<a href="https://colab.research.google.com/github/auth-create/DDfiles/blob/main/engenharia_reversa_videos_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CÉLULA 1: SETUP INICIAL - INSTALAÇÃO DE DEPENDÊNCIAS
!pip install -q opencv-python
!pip install -q pytesseract
!pip install -q pydub
!pip install -q SpeechRecognition
!pip install -q librosa
!pip install -q openpyxl
!pip install -q pandas

# Instalar Tesseract OCR
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-por

print("✅ Todas as dependências foram instaladas com sucesso!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 50.1 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,782 kB]
Get:13 https://r2u.stat.il

In [ ]:
# Cole este código em uma célula NOVA no início do seu Colab:
!pip install opencv-python librosa speechrecognition pytesseract textblob scikit-learn matplotlib seaborn

# Execute apenas UMA VEZ para instalar as dependências

In [ ]:
# ============================================================================
# SISTEMA MODULAR DE ENGENHARIA REVERSA - GOOGLE COLAB
# Cada célula = Um processo independente com validação de pré-requisitos
# ============================================================================

# ============================================================================
# CÉLULA 1: SETUP INICIAL E DEPENDÊNCIAS
# ============================================================================

# Instalar dependências necessárias
!pip install -q moviepy librosa pytesseract opencv-python pandas openpyxl matplotlib seaborn pillow
!apt-get update -qq && apt-get install -y -qq tesseract-ocr tesseract-ocr-por ffmpeg

# Imports necessários
import os
import json
import pandas as pd
from datetime import datetime
import logging
import cv2
import numpy as np
import pytesseract
import librosa
from moviepy.editor import VideoFileClip
import matplotlib.pyplot as plt
from PIL import Image
from collections import Counter
import seaborn as sns
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

# Montar Google Drive
drive.mount('/content/drive')

print("✅ SETUP CONCLUÍDO")
print("Dependências instaladas e Google Drive montado.")
print("Próximo passo: Execute a CÉLULA 2 - CONFIGURAÇÃO INICIAL")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ SETUP CONCLUÍDO
Dependências instaladas e Google Drive montado.
Próximo passo: Execute a CÉLULA 2 - CONFIGURAÇÃO INICIAL


In [ ]:
# CÉLULA 2: CONFIGURAÇÃO INICIAL E VALIDAÇÃO DA PASTA
# ============================================================================

# CONFIGURE SEU CAMINHO AQUI
CAMINHO_PASTA_VIDEOS = "/content/drive/MyDrive/Videos Dona Done"

# Classe para gerenciar configuração inicial
class ConfiguradorProjeto:
    def __init__(self, caminho_pasta):
        self.pasta_videos = self._validar_caminho(caminho_pasta)
        self.pasta_trabalho = os.path.join(self.pasta_videos, "_engenharia_reversa")
        self._criar_estrutura()
        self._configurar_logging()

    def _validar_caminho(self, caminho):
        if caminho == "/content/drive/MyDrive/SUBSTITUA_PELO_SEU_CAMINHO":
            raise ValueError("❌ ERRO: Você precisa alterar CAMINHO_PASTA_VIDEOS com o caminho real da sua pasta")

        if not os.path.exists(caminho):
            raise ValueError(f"❌ ERRO: Pasta não encontrada: {caminho}")

        return caminho

    def _criar_estrutura(self):
        estrutura = [
            'config', 'logs', 'dados', 'frames_extraidos',
            'analise_texto', 'analise_audio', 'capturas',
            'relatorios', 'blueprint', 'temp'
        ]

        os.makedirs(self.pasta_trabalho, exist_ok=True)
        for pasta in estrutura:
            os.makedirs(os.path.join(self.pasta_trabalho, pasta), exist_ok=True)

    def _configurar_logging(self):
        log_file = os.path.join(self.pasta_trabalho, "logs", f"sistema_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[logging.FileHandler(log_file, encoding='utf-8')]
        )
        self.logger = logging.getLogger(__name__)

    def salvar_configuracao(self):
        config = {
            "projeto": {
                "pasta_videos": self.pasta_videos,
                "pasta_trabalho": self.pasta_trabalho,
                "criado_em": datetime.now().isoformat(),
                "versao": "modular_v1.0"
            },
            "status_etapas": {
                "configuracao": True,
                "descoberta_videos": False,
                "metadados": False,
                "decomposicao": False,
                "analise_padroes": False,
                "blueprint": False
            }
        }

        config_path = os.path.join(self.pasta_trabalho, "config", "config.json")
        with open(config_path, 'w', encoding='utf-8') as f:
            json.dump(config, f, indent=2, ensure_ascii=False)

        return config_path

# Executar configuração
try:
    configurador = ConfiguradorProjeto(CAMINHO_PASTA_VIDEOS)
    config_path = configurador.salvar_configuracao()

    print("✅ CONFIGURAÇÃO CONCLUÍDA")
    print(f"Pasta de trabalho criada: {configurador.pasta_trabalho}")
    print(f"Configuração salva: {config_path}")
    print("Próximo passo: Execute a CÉLULA 3 - DESCOBERTA DE VÍDEOS")

    # Salvar variáveis globais para próximas células
    PASTA_VIDEOS = configurador.pasta_videos
    PASTA_TRABALHO = configurador.pasta_trabalho

except Exception as e:
    print(f"❌ ERRO NA CONFIGURAÇÃO: {e}")
    print("Corrija o erro antes de prosseguir.")

✅ CONFIGURAÇÃO CONCLUÍDA
Pasta de trabalho criada: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa
Configuração salva: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/config/config.json
Próximo passo: Execute a CÉLULA 3 - DESCOBERTA DE VÍDEOS


In [ ]:
# CÉLULA 3: DESCOBERTA E CATALOGAÇÃO DE VÍDEOS
# ============================================================================

def verificar_prerequisito_etapa2():
    """Verifica se a configuração foi executada com sucesso"""
    try:
        if not 'PASTA_TRABALHO' in globals():
            raise Exception("Execute a CÉLULA 2 primeiro")

        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        if not os.path.exists(config_path):
            raise Exception("Configuração não encontrada. Execute a CÉLULA 2 primeiro")

        return True
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO: {e}")
        return False

def descobrir_catalogar_videos():
    """Descobre e cataloga todos os vídeos na pasta"""
    formatos_aceitos = ['.mp4', '.mov', '.avi', '.mkv', '.webm', '.m4v']
    videos_encontrados = []

    print("Descobrindo vídeos...")

    for root, dirs, files in os.walk(PASTA_VIDEOS):
        if "_engenharia_reversa" in root:
            continue

        for file in files:
            if any(file.lower().endswith(fmt) for fmt in formatos_aceitos):
                video_path = os.path.join(root, file)

                try:
                    stat_info = os.stat(video_path)
                    video_id = f"vid_{len(videos_encontrados)+1:03d}"

                    video_info = {
                        'id': video_id,
                        'nome_arquivo': file,
                        'caminho_completo': video_path,
                        'caminho_relativo': os.path.relpath(video_path, PASTA_VIDEOS),
                        'tamanho_mb': round(stat_info.st_size / (1024*1024), 2),
                        'data_modificacao': datetime.fromtimestamp(stat_info.st_mtime).isoformat(),
                        'extensao': os.path.splitext(file)[1].lower(),
                        'status': 'descoberto'
                    }

                    videos_encontrados.append(video_info)

                except Exception as e:
                    continue

    return videos_encontrados

def salvar_lista_videos(videos):
    """Salva lista de vídeos encontrados"""
    videos_path = os.path.join(PASTA_TRABALHO, "dados", "videos_descobertos.json")
    with open(videos_path, 'w', encoding='utf-8') as f:
        json.dump(videos, f, indent=2, ensure_ascii=False)

    # Atualizar status no config
    config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
    with open(config_path, 'r', encoding='utf-8') as f:
        config = json.load(f)

    config['status_etapas']['descoberta_videos'] = True
    config['total_videos_encontrados'] = len(videos)

    with open(config_path, 'w', encoding='utf-8') as f:
        json.dump(config, f, indent=2, ensure_ascii=False)

    return videos_path

# Executar descoberta
if verificar_prerequisito_etapa2():
    try:
        videos_encontrados = descobrir_catalogar_videos()

        if not videos_encontrados:
            print("❌ NENHUM VÍDEO ENCONTRADO")
            print(f"Verifique se há vídeos na pasta: {PASTA_VIDEOS}")
        else:
            videos_path = salvar_lista_videos(videos_encontrados)

            print("✅ DESCOBERTA DE VÍDEOS CONCLUÍDA")
            print(f"Vídeos encontrados: {len(videos_encontrados)}")
            print(f"Lista salva em: {videos_path}")

            # Mostrar resumo
            extensoes = Counter([v['extensao'] for v in videos_encontrados])
            print(f"Formatos encontrados: {dict(extensoes)}")
            print("Próximo passo: Execute a CÉLULA 4 - EXTRAÇÃO DE METADADOS")

    except Exception as e:
        print(f"❌ ERRO NA DESCOBERTA: {e}")

Descobrindo vídeos...
✅ DESCOBERTA DE VÍDEOS CONCLUÍDA
Vídeos encontrados: 40
Lista salva em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/dados/videos_descobertos.json
Formatos encontrados: {'.mp4': 40}
Próximo passo: Execute a CÉLULA 4 - EXTRAÇÃO DE METADADOS


In [ ]:
# CÉLULA 4: EXTRAÇÃO DE METADADOS DOS VÍDEOS
# ============================================================================

def verificar_prerequisito_etapa3():
    """Verifica se a descoberta foi executada com sucesso"""
    try:
        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        if not config['status_etapas']['descoberta_videos']:
            raise Exception("Execute a CÉLULA 3 primeiro")

        videos_path = os.path.join(PASTA_TRABALHO, "dados", "videos_descobertos.json")
        if not os.path.exists(videos_path):
            raise Exception("Lista de vídeos não encontrada. Execute a CÉLULA 3 primeiro")

        return True, videos_path
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO: {e}")
        return False, None

def extrair_metadados_video(video_info):
    """Extrai metadados técnicos de um vídeo"""
    video_path = video_info['caminho_completo']
    video_id = video_info['id']

    # Análise com OpenCV
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise Exception("Não foi possível abrir o vídeo")

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    largura = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    altura = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duracao = frame_count / fps if fps > 0 else 0

    # Capturar primeiro frame
    ret, primeiro_frame = cap.read()
    cap.release()

    # Análise de áudio
    try:
        clip = VideoFileClip(video_path)
        tem_audio = clip.audio is not None
        clip.close()
    except:
        tem_audio = False

    # Análise do primeiro frame
    analise_frame = {}
    if ret:
        # Salvar primeiro frame
        capturas_dir = os.path.join(PASTA_TRABALHO, "capturas")
        frame_path = os.path.join(capturas_dir, f"{video_id}_primeiro_frame.jpg")
        cv2.imwrite(frame_path, primeiro_frame)

        # Análises do frame
        gray = cv2.cvtColor(primeiro_frame, cv2.COLOR_BGR2GRAY)
        complexidade = cv2.Laplacian(gray, cv2.CV_64F).var()
        brilho = np.mean(gray)

        analise_frame = {
            'path': frame_path,
            'complexidade_visual': float(complexidade),
            'brilho_medio': float(brilho),
            'tem_muito_texto': bool(complexidade > 500),
            'e_escuro': bool(brilho < 100),
            'e_claro': bool(brilho > 200)
        }

    # Detectar formato
    ratio = largura / altura if altura > 0 else 0
    if 0.5 <= ratio <= 0.6:
        formato = "vertical_9_16" if altura > largura * 1.5 else "vertical_4_5"
    elif 0.8 <= ratio <= 1.2:
        formato = "quadrado_1_1"
    elif ratio >= 1.3:
        formato = "horizontal_16_9"
    else:
        formato = "personalizado"

    # Compilar metadados - converter todos os valores para tipos básicos Python
    metadados = {
        **video_info,
        'duracao_segundos': float(duracao),
        'fps': float(fps),
        'largura': int(largura),
        'altura': int(altura),
        'resolucao': f"{largura}x{altura}",
        'aspect_ratio': float(ratio),
        'total_frames': int(frame_count),
        'tem_audio': bool(tem_audio),
        'formato_detectado': str(formato),
        'primeiro_frame': analise_frame,
        'data_analise': datetime.now().isoformat()
    }

    return metadados

def processar_metadados_todos_videos():
    """Processa metadados de todos os vídeos"""
    # Carregar lista de vídeos
    videos_path = os.path.join(PASTA_TRABALHO, "dados", "videos_descobertos.json")
    with open(videos_path, 'r', encoding='utf-8') as f:
        videos_lista = json.load(f)

    metadados_completos = []
    sucessos = 0

    print(f"Processando metadados de {len(videos_lista)} vídeos...")

    for i, video in enumerate(videos_lista, 1):
        print(f"[{i}/{len(videos_lista)}] {video['nome_arquivo']}")

        try:
            metadados = extrair_metadados_video(video)
            metadados_completos.append(metadados)
            video['status'] = 'metadados_extraidos'
            sucessos += 1

            # Mostrar progresso
            duracao = metadados['duracao_segundos']
            formato = metadados['formato_detectado']
            print(f"  ✓ {duracao:.1f}s | {formato} | Audio: {'Sim' if metadados['tem_audio'] else 'Não'}")

        except Exception as e:
            print(f"  ❌ Erro: {e}")
            video['status'] = 'erro_metadados'
            continue

    return metadados_completos, sucessos

def converter_para_json_serializable(obj):
    """Converte objetos para formato JSON serializable"""
    if isinstance(obj, dict):
        return {key: converter_para_json_serializable(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [converter_para_json_serializable(item) for item in obj]
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, (np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.float64, np.float32)):
        return float(obj)
    elif isinstance(obj, np.bool_):
        return bool(obj)
    else:
        return obj

def salvar_metadados(metadados):
    """Salva metadados processados"""
    # Converter dados para formato JSON serializable
    metadados_serializaveis = converter_para_json_serializable(metadados)

    # Salvar JSON
    metadados_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
    with open(metadados_path, 'w', encoding='utf-8') as f:
        json.dump(metadados_serializaveis, f, indent=2, ensure_ascii=False)

    # Salvar Excel
    df = pd.DataFrame(metadados)
    excel_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_videos.xlsx")
    df.to_excel(excel_path, index=False)

    # Atualizar config
    config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
    with open(config_path, 'r', encoding='utf-8') as f:
        config = json.load(f)

    config['status_etapas']['metadados'] = True
    config['videos_com_metadados'] = len(metadados)

    with open(config_path, 'w', encoding='utf-8') as f:
        json.dump(config, f, indent=2, ensure_ascii=False)

    return metadados_path, excel_path

# Executar extração de metadados
prerequisito_ok, videos_path = verificar_prerequisito_etapa3()

if prerequisito_ok:
    try:
        metadados, sucessos = processar_metadados_todos_videos()

        if sucessos == 0:
            print("❌ NENHUM VÍDEO PROCESSADO COM SUCESSO")
        else:
            json_path, excel_path = salvar_metadados(metadados)

            print("✅ EXTRAÇÃO DE METADADOS CONCLUÍDA")
            print(f"Vídeos processados: {sucessos}")
            print(f"Dados salvos em: {json_path}")
            print(f"Excel gerado: {excel_path}")

            # Resumo
            formatos = Counter([m['formato_detectado'] for m in metadados])
            duracoes = [m['duracao_segundos'] for m in metadados]

            print(f"\nResumo:")
            print(f"Formatos: {dict(formatos)}")
            print(f"Duração média: {np.mean(duracoes):.1f}s")
            print(f"Vídeos com áudio: {sum(1 for m in metadados if m['tem_audio'])}")
            print("Próximo passo: Execute a CÉLULA 5 - DECOMPOSIÇÃO DE VÍDEOS")

    except Exception as e:
        print(f"❌ ERRO NA EXTRAÇÃO DE METADADOS: {e}")

Processando metadados de 40 vídeos...
[1/40] entenda mais sobre a consciencia cristica.mp4
  ✓ 17.8s | vertical_9_16 | Audio: Sim
[2/40] voce quer prosperar.mp4
  ✓ 52.9s | vertical_9_16 | Audio: Sim
[3/40] Por que é que muitos cristãos não prosperam financeiramente_ não acessam uma ruptura financeira.mp4
  ✓ 93.3s | vertical_9_16 | Audio: Sim
[4/40] como é que eu sei que algo que eu estou carregando é da parte de Deus Por dois motivos..mp4
  ✓ 59.1s | vertical_9_16 | Audio: Sim
[5/40] ja perdoei mas nao quero olhar para cara dela.mp4
  ✓ 62.5s | vertical_9_16 | Audio: Sim
[6/40] Não perca tempo batendo boca com os outros.mp4
  ✓ 13.9s | vertical_9_16 | Audio: Sim
[7/40] Pare de fazer isso agora_ isso só te atrapalha.mp4
  ✓ 24.3s | vertical_9_16 | Audio: Sim
[8/40] E você não tem que ficar focado em ter uma vida normal_ você tem que ter o foco para investir em você.mp4
  ✓ 46.2s | vertical_9_16 | Audio: Sim
[9/40] Se você quiser explodir na vida_ você vai organizar a sua

In [ ]:
# ============================================================================
# CÉLULA 4.1: SUBLAYER 2.3 - OCR PARA LEGENDAS, EMOJIS E OVERLAYS
# Esta célula deve ser executada APÓS a CÉLULA 4 (Extração de Metadados).
# ============================================================================

def verificar_prerequisito_etapa4():
    """Verifica se a extração de metadados foi executada com sucesso"""
    try:
        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        if not config['status_etapas']['metadados']:
            raise Exception("Execute a CÉLULA 4 primeiro")

        videos_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
        if not os.path.exists(videos_path):
            raise Exception("Metadados de vídeos não encontrados. Execute a CÉLULA 4 primeiro")

        return True, videos_path
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO: {e}")
        return False, None

def extrair_texto_ocr(image_path):
    """Extrai texto de uma imagem usando OCR."""
    try:
        img = Image.open(image_path)
        # Adicionar pré-processamento básico para melhorar o OCR
        img = img.convert('L') # Converte para escala de cinza
        img = Image.fromarray(cv2.threshold(np.array(img), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]) # Binarização
        text = pytesseract.image_to_string(img, lang='por+eng') # Tenta português e inglês
        return text.strip()
    except Exception as e:
        logging.error(f"Erro ao extrair OCR da imagem {image_path}: {e}")
        return ""

def processar_ocr_videos(videos_info):
    """Processa OCR para todos os vídeos e salva os resultados."""
    analise_texto_dir = os.path.join(PASTA_TRABALHO, "analise_texto")
    os.makedirs(analise_texto_dir, exist_ok=True)

    for video_info in videos_info:
        video_id = video_info['id']
        video_path = video_info['caminho_completo']
        logging.info(f"Iniciando OCR para o vídeo: {video_id}")

        ocr_results = []
        temp_frames_dir = os.path.join(PASTA_TRABALHO, "temp", video_id, "frames")
        os.makedirs(temp_frames_dir, exist_ok=True)

        # Extrair frames a cada segundo para OCR
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = frame_count / fps if fps > 0 else 0

        if fps == 0: # Evita divisão por zero
            logging.warning(f"FPS do vídeo {video_id} é zero. Pulando OCR de frames.")
            cap.release()
            continue

        for i in range(int(duration) + 1):
            frame_time_sec = i
            cap.set(cv2.CAP_PROP_POS_MSEC, frame_time_sec * 1000)
            ret, frame = cap.read()
            if ret:
                frame_filename = os.path.join(temp_frames_dir, f"frame_{frame_time_sec:04d}.jpg")
                cv2.imwrite(frame_filename, frame)
                extracted_text = extrair_texto_ocr(frame_filename)
                if extracted_text:
                    ocr_results.append({
                        'video_id': video_id,
                        'timestamp_sec': frame_time_sec,
                        'texto_extraido': extracted_text
                    })
            else:
                break
        cap.release()

        # Salvar resultados do OCR para o vídeo atual
        output_json_path = os.path.join(analise_texto_dir, f"{video_id}_ocr_results.json")
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(ocr_results, f, indent=2, ensure_ascii=False)
        logging.info(f"OCR para {video_id} concluído. Resultados salvos em {output_json_path}")

        # Limpar frames temporários
        if os.path.exists(temp_frames_dir):
            import shutil
            shutil.rmtree(temp_frames_dir)

    # Atualizar status no config (opcional, pode ser feito na célula pai)
    # config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
    # with open(config_path, 'r', encoding='utf-8') as f:
    #     config = json.load(f)
    # config['status_etapas']['ocr_texto'] = True # Adicionar esta etapa ao config
    # with open(config_path, 'w', encoding='utf-8') as f:
    #     json.dump(config, f, indent=2, ensure_ascii=False)

    print("✅ SUBLAYER 2.3 (OCR) CONCLUÍDA")
    print(f"Resultados do OCR salvos em: {analise_texto_dir}")

# Executar Sublayer 2.3
if __name__ == '__main__':
    # Assegurar que PASTA_TRABALHO e PASTA_VIDEOS estejam definidas (simulando ambiente Colab)
    # No Google Colab, estas variáveis já estariam definidas pela CÉLULA 2
    # Para teste local, defina-as aqui:
    # PASTA_TRABALHO = "/content/drive/MyDrive/Videos Dona Done/_engenharia_reversa"
    # PASTA_VIDEOS = "/content/drive/MyDrive/Videos Dona Done"

    prereq_ok, videos_json_path = verificar_prerequisito_etapa4()
    if prereq_ok:
        try:
            with open(videos_json_path, 'r', encoding='utf-8') as f:
                videos_data = json.load(f)
            processar_ocr_videos(videos_data)
        except Exception as e:
            print(f"❌ ERRO NA SUBLAYER 2.3 (OCR): {e}")
            logging.error(f"Erro na Sublayer 2.3 (OCR): {e}")

KeyboardInterrupt: 

In [ ]:

# ============================================================================
# CÉLULA 4.2: SUBLAYER 3.1 - TRANSCRIÇÃO DE ÁUDIO (SPEECH-TO-TEXT)
# Esta célula deve ser executada APÓS a CÉLULA 4 (Extração de Metadados).
# ============================================================================

# Importar bibliotecas necessárias (certifique-se de que estejam instaladas na CÉLULA 1)
import os
import json
import logging
from moviepy.editor import VideoFileClip

# Para transcrição de áudio, usaremos a biblioteca SpeechRecognition e PyAudio.
# PyAudio pode ser um pouco complicado de instalar no Colab, então vamos usar uma abordagem mais simples
# que extrai o áudio e usa uma API gratuita ou uma biblioteca offline mais leve.
# Para este exemplo, vamos simular a transcrição ou usar uma biblioteca simples.
# Para uma solução robusta, você pode integrar com Google Speech-to-Text API, AssemblyAI, etc.
# Por simplicidade e para evitar dependências externas complexas, vamos usar uma abordagem básica com pydub e SpeechRecognition.
# Certifique-se de que `pydub` e `SpeechRecognition` estejam instalados na CÉLULA 1.
# !pip install -q pydub SpeechRecognition

# Adicionar import para SpeechRecognition
import speech_recognition as sr
from pydub import AudioSegment
from pydub.silence import split_on_silence

def verificar_prerequisito_etapa4_audio():
    """Verifica se a extração de metadados foi executada com sucesso"""
    try:
        # Verifica se as variáveis globais estão definidas
        if not (
            'PASTA_TRABALHO' in globals() and
            'PASTA_VIDEOS' in globals()
        ):
            raise Exception("As variáveis globais PASTA_TRABALHO e PASTA_VIDEOS não estão definidas. Execute a CÉLULA 2 primeiro.")

        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        if not config['status_etapas']['metadados']:
            raise Exception("A etapa de metadados não foi concluída. Execute a CÉLULA 4 primeiro.")

        videos_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
        if not os.path.exists(videos_path):
            raise Exception("Metadados de vídeos não encontrados. Execute a CÉLULA 4 primeiro.")

        return True, videos_path
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO PARA TRANSCRIÇÃO DE ÁUDIO: {e}")
        logging.error(f"Pré-requisito não atendido para transcrição de áudio: {e}")
        return False, None

def transcrever_audio(audio_path, video_id):
    """Transcreve o áudio de um arquivo usando SpeechRecognition."""
    recognizer = sr.Recognizer()
    transcricao_completa = []

    try:
        # Carregar o arquivo de áudio
        audio = AudioSegment.from_wav(audio_path)

        # Dividir o áudio em pedaços baseados no silêncio
        # Isso ajuda a transcrever áudios longos e melhora a precisão
        chunks = split_on_silence(audio,
                                  min_silence_len=500, # milissegundos de silêncio para considerar uma pausa
                                  silence_thresh=-40) # limite de decibéis para considerar silêncio

        # Criar um diretório temporário para os pedaços de áudio
        temp_audio_chunks_dir = os.path.join(PASTA_TRABALHO, "temp", video_id, "audio_chunks")
        os.makedirs(temp_audio_chunks_dir, exist_ok=True)

        for i, chunk in enumerate(chunks):
            chunk_filename = os.path.join(temp_audio_chunks_dir, f"chunk_{i}.wav")
            chunk.export(chunk_filename, format="wav")

            with sr.AudioFile(chunk_filename) as source:
                audio_listened = recognizer.record(source)
                try:
                    # Usar o reconhecedor de fala do Google (online)
                    text = recognizer.recognize_google(audio_listened, language="pt-BR")
                    transcricao_completa.append(text)
                    logging.info(f"  Chunk {i} transcrito: {text[:50]}...")
                except sr.UnknownValueError:
                    logging.warning(f"  Não foi possível entender o áudio no chunk {i}.")
                except sr.RequestError as e:
                    logging.error(f"  Erro na requisição ao serviço de fala do Google no chunk {i}: {e}")

        # Limpar arquivos temporários
        import shutil
        shutil.rmtree(temp_audio_chunks_dir)

    except Exception as e:
        logging.error(f"Erro ao processar áudio para transcrição {audio_path}: {e}")
        return ""

    return " ".join(transcricao_completa)

def processar_transcricao_audio_videos(videos_info):
    """Extrai áudio dos vídeos e realiza a transcrição, salvando os resultados."""
    transcricoes_audio_dir = os.path.join(PASTA_TRABALHO, "analise_audio", "transcricoes")
    os.makedirs(transcricoes_audio_dir, exist_ok=True)

    for i, video_info in enumerate(videos_info):
        video_id = video_info["id"]
        video_path = video_info["caminho_completo"]

        print(f"\nProgresso: {i+1}/{len(videos_info)} - Iniciando transcrição de áudio para o vídeo: {video_id} ({video_info["nome_arquivo"]})")
        logging.info(f"Iniciando transcrição de áudio para o vídeo: {video_id}")

        audio_output_path = os.path.join(PASTA_TRABALHO, "temp", f"{video_id}_audio.wav")
        os.makedirs(os.path.dirname(audio_output_path), exist_ok=True)

        try:
            # Extrair áudio do vídeo
            clip = VideoFileClip(video_path)
            if clip.audio:
                clip.audio.write_audiofile(audio_output_path, verbose=False, logger=None)
                clip.close()
                logging.info(f"Áudio extraído para: {audio_output_path}")

                # Transcrever o áudio
                transcricao = transcrever_audio(audio_output_path, video_id)

                # Salvar transcrição
                output_txt_path = os.path.join(transcricoes_audio_dir, f"{video_id}_transcricao.txt")
                with open(output_txt_path, 'w', encoding='utf-8') as f:
                    f.write(transcricao)
                logging.info(f"Transcrição para {video_id} concluída e salva em: {output_txt_path}")
                print(f"  ✅ Transcrição concluída para {video_id}. Texto salvo em: {output_txt_path}")
                print(f"  Primeiras 200 caracteres da transcrição: {transcricao[:200]}...")
            else:
                logging.warning(f"Vídeo {video_id} não possui trilha de áudio. Pulando transcrição.")
                print(f"  ⚠️ Vídeo {video_id} não possui trilha de áudio. Pulando transcrição.")

        except Exception as e:
            logging.error(f"Erro ao extrair ou transcrever áudio do vídeo {video_id}: {e}")
            print(f"  ❌ ERRO ao processar áudio do vídeo {video_id}: {e}")
        finally:
            # Limpar arquivo de áudio temporário
            if os.path.exists(audio_output_path):
                os.remove(audio_output_path)
                logging.info(f"Arquivo temporário de áudio removido: {audio_output_path}")

    print("\n✅ SUBLAYER 3.1 (TRANSCRIÇÃO DE ÁUDIO) CONCLUÍDA")
    print(f"Todas as transcrições de áudio salvas em: {transcricoes_audio_dir}")

# ============================================================================
# INSTRUÇÕES DE EXECUÇÃO NO GOOGLE COLAB:
# 1. Certifique-se de que a CÉLULA 1 (Setup Inicial) foi executada e instalou
#    as dependências `pydub` e `SpeechRecognition`.
#    Adicione as seguintes linhas na CÉLULA 1, se ainda não estiverem lá:
#    `!pip install -q pydub SpeechRecognition`
# 2. Certifique-se de que a CÉLULA 2 (Configuração Inicial) foi executada e
#    definiu as variáveis globais `PASTA_TRABALHO` e `PASTA_VIDEOS`.
# 3. Certifique-se de que a CÉLULA 4 (Extração de Metadados) foi executada e
#    gerou o arquivo `metadados_completos.json`.
# 4. Copie e cole TODO o conteúdo desta célula em uma NOVA CÉLULA no seu
#    notebook Google Colab, logo APÓS a CÉLULA 4.
# 5. Execute esta nova célula.
# ============================================================================

# Bloco de execução principal para o Colab
if __name__ == '__main__':
    prereq_ok, videos_json_path = verificar_prerequisito_etapa4_audio()
    if prereq_ok:
        try:
            with open(videos_json_path, 'r', encoding='utf-8') as f:
                videos_data = json.load(f)
            processar_transcricao_audio_videos(videos_data)
        except Exception as e:
            print(f"❌ ERRO GERAL NA SUBLAYER 3.1 (TRANSCRIÇÃO DE ÁUDIO): {e}")
            logging.error(f"Erro geral na Sublayer 3.1 (Transcrição de Áudio): {e}")


Progresso: 1/40 - Iniciando transcrição de áudio para o vídeo: vid_001 (entenda mais sobre a consciencia cristica.mp4)
  ✅ Transcrição concluída para vid_001. Texto salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_audio/transcricoes/vid_001_transcricao.txt
  Primeiras 200 caracteres da transcrição: a consciência crítica é um estado elevado é o estado de percepção espiritual mais elevado onde você vai viver em Total alinhamento os princípios divinos do Amor Incondicional da compaixão da unidade d...

Progresso: 2/40 - Iniciando transcrição de áudio para o vídeo: vid_002 (voce quer prosperar.mp4)
  ✅ Transcrição concluída para vid_002. Texto salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_audio/transcricoes/vid_002_transcricao.txt
  Primeiras 200 caracteres da transcrição: você quer prosperar sim ou não quando você falar que vai fazer alguma coisa dê tudo que você tem que isso acontecer tudo que eu começo eu falo assim não sei c

  ✅ Transcrição concluída para vid_004. Texto salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_audio/transcricoes/vid_004_transcricao.txt
  Primeiras 200 caracteres da transcrição: Camila como é que eu sei que é algo que eu estou carregando é da parte de Deus por dois motivos primeiro quando você quer se desfazer você não consegue é a maior prova de que não foi tua mão que plant...

Progresso: 5/40 - Iniciando transcrição de áudio para o vídeo: vid_005 (ja perdoei mas nao quero olhar para cara dela.mp4)
  ✅ Transcrição concluída para vid_005. Texto salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_audio/transcricoes/vid_005_transcricao.txt
  Primeiras 200 caracteres da transcrição: já perdoei mas não quero olhar para a cara dela não perdoa já perdoei Mas não quero que passe na minha frente não perdoou já perdoei para mim é igual essa parede nem fede nem cheira não perdoou não pe...

Progresso: 6/40 - Iniciando transcrição de áudi

  ✅ Transcrição concluída para vid_020. Texto salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_audio/transcricoes/vid_020_transcricao.txt
  Primeiras 200 caracteres da transcrição: no fundo no fundo Ninguém te ama porque o amor tudo sofre tudo crê tudo suporta não se compadece do mal não se alegra na injustiça é maior que profecia é maior que tudo certeza que você ama o aperfeiç...

Progresso: 21/40 - Iniciando transcrição de áudio para o vídeo: vid_021 (O circulo de pessoa ao seu redor importa.mp4)


  ✅ Transcrição concluída para vid_021. Texto salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_audio/transcricoes/vid_021_transcricao.txt
  Primeiras 200 caracteres da transcrição: ...

Progresso: 22/40 - Iniciando transcrição de áudio para o vídeo: vid_022 (ate quando voce vai ficar culpando os outros.mp4)
  ✅ Transcrição concluída para vid_022. Texto salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_audio/transcricoes/vid_022_transcricao.txt
  Primeiras 200 caracteres da transcrição: tua pergunta é até quando você vai ficar no lugar de vítima buscando culpados pela situação que você se encontra porque enquanto você culpa alguém ou alguma coisa ou uma situação ou uma pessoa aquela ...

Progresso: 23/40 - Iniciando transcrição de áudio para o vídeo: vid_023 (Se não for para produzir_ tira o celular da sua mão e não toque nele..mp4)
  ✅ Transcrição concluída para vid_023. Texto salvo em: /content/drive/MyDrive/Videos Dona Done/

  ✅ Transcrição concluída para vid_026. Texto salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_audio/transcricoes/vid_026_transcricao.txt
  Primeiras 200 caracteres da transcrição: ...

Progresso: 27/40 - Iniciando transcrição de áudio para o vídeo: vid_027 (Então_ como é que reprograma o cérebro.mp4)
  ✅ Transcrição concluída para vid_027. Texto salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_audio/transcricoes/vid_027_transcricao.txt
  Primeiras 200 caracteres da transcrição: Então como é que reprograma o cérebro eu vou te ensinar você acabou de nascer vídeo vai entregar para família do Tarzan se você andar com o Tarzan a Jana ou janja e pegar e andar com esse povo e viver...

Progresso: 28/40 - Iniciando transcrição de áudio para o vídeo: vid_028 (Você quer saber como é que faz pra ter um cérebro idiota agrade tudo que o seu cérebro pede pra você fazer.mp4)
  ✅ Transcrição concluída para vid_028. Texto salvo em: /con

  ✅ Transcrição concluída para vid_034. Texto salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_audio/transcricoes/vid_034_transcricao.txt
  Primeiras 200 caracteres da transcrição: eu quero te mostrar uma coisa e sujando feio tá bagunça mas ninguém tá dizendo que...

Progresso: 35/40 - Iniciando transcrição de áudio para o vídeo: vid_035 (produzir não é trabalhar demais.mp4)


  ✅ Transcrição concluída para vid_035. Texto salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_audio/transcricoes/vid_035_transcricao.txt
  Primeiras 200 caracteres da transcrição: O que é produzir não é trabalhar demais tá produzir aderência aquilo que eu ajustei vou se eu falo que vou trabalhar 2 horas por dia e trabalho 2 horas por dia eu estou 100% Ao contrário de você que f...

Progresso: 36/40 - Iniciando transcrição de áudio para o vídeo: vid_036 (tenha amor proprio e alta confianca.mp4)
  ✅ Transcrição concluída para vid_036. Texto salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_audio/transcricoes/vid_036_transcricao.txt
  Primeiras 200 caracteres da transcrição: restaurar posso inclusive Nós temos muitas técnicas para isso Se esse for o teu objetivo mas talvez momento em que você restaura um pingo de amor próprio e você entende que a motivação de querer resta...

Progresso: 37/40 - Iniciando transcrição de áudio para o

In [ ]:

# ============================================================================
# CÉLULA 4.3: SUBLAYER 3.2 - DETECÇÃO DE PICOS DE ÁUDIO E SINCRONIZAÇÃO
# Esta célula deve ser executada APÓS a CÉLULA 4 (Extração de Metadados).
# ============================================================================

# Importar bibliotecas necessárias (certifique-se de que estejam instaladas na CÉLULA 1)
import os
import json
import logging
import librosa
import numpy as np
from moviepy.editor import VideoFileClip

def verificar_prerequisito_etapa4_audio_peaks():
    """Verifica se a extração de metadados foi executada com sucesso"""
    try:
        # Verifica se as variáveis globais estão definidas
        if not (
            'PASTA_TRABALHO' in globals() and
            'PASTA_VIDEOS' in globals()
        ):
            raise Exception("As variáveis globais PASTA_TRABALHO e PASTA_VIDEOS não estão definidas. Execute a CÉLULA 2 primeiro.")

        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        if not config['status_etapas']['metadados']:
            raise Exception("A etapa de metadados não foi concluída. Execute a CÉLULA 4 primeiro.")

        videos_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
        if not os.path.exists(videos_path):
            raise Exception("Metadados de vídeos não encontrados. Execute a CÉLULA 4 primeiro.")

        return True, videos_path
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO PARA DETECÇÃO DE PICOS DE ÁUDIO: {e}")
        logging.error(f"Pré-requisito não atendido para detecção de picos de áudio: {e}")
        return False, None

def detectar_picos_audio(audio_path, sr_rate=22050):
    """Detecta picos de áudio em um arquivo de áudio."""
    try:
        y, sr = librosa.load(audio_path, sr=sr_rate)
        # Detecta picos de áudio usando a função onset_detect da librosa
        # onset_env: função de detecção de onset (início de um som)
        # picos: índices dos picos detectados
        onset_env = librosa.onset.onset_detect(y=y, sr=sr, units='time')
        return onset_env.tolist() # Retorna os tempos dos picos em segundos
    except Exception as e:
        logging.error(f"Erro ao detectar picos de áudio em {audio_path}: {e}")
        return []

def processar_picos_audio_videos(videos_info):
    """Processa a detecção de picos de áudio para todos os vídeos e salva os resultados."""
    analise_audio_dir = os.path.join(PASTA_TRABALHO, "analise_audio")
    os.makedirs(analise_audio_dir, exist_ok=True)

    resultados_picos_audio = {}

    for i, video_info in enumerate(videos_info):
        video_id = video_info["id"]
        video_path = video_info["caminho_completo"]

        print(f"\nProgresso: {i+1}/{len(videos_info)} - Iniciando detecção de picos de áudio para o vídeo: {video_id} ({video_info["nome_arquivo"]})")
        logging.info(f"Iniciando detecção de picos de áudio para o vídeo: {video_id}")

        audio_output_path = os.path.join(PASTA_TRABALHO, "temp", f"{video_id}_audio_temp.wav")
        os.makedirs(os.path.dirname(audio_output_path), exist_ok=True)

        try:
            # Extrair áudio do vídeo
            clip = VideoFileClip(video_path)
            if clip.audio:
                clip.audio.write_audiofile(audio_output_path, verbose=False, logger=None)
                clip.close()
                logging.info(f"Áudio extraído para: {audio_output_path}")

                # Detectar picos de áudio
                picos_tempo = detectar_picos_audio(audio_output_path)
                resultados_picos_audio[video_id] = picos_tempo

                output_json_path = os.path.join(analise_audio_dir, f"{video_id}_picos_audio.json")
                with open(output_json_path, 'w', encoding='utf-8') as f:
                    json.dump({"video_id": video_id, "picos_audio_segundos": picos_tempo}, f, indent=2, ensure_ascii=False)
                logging.info(f"Picos de áudio para {video_id} salvos em: {output_json_path}")
                print(f"  ✅ Detecção de picos de áudio concluída para {video_id}. Encontrados {len(picos_tempo)} picos.")
                print(f"  Picos (primeiros 10): {picos_tempo[:10]}...")
            else:
                logging.warning(f"Vídeo {video_id} não possui trilha de áudio. Pulando detecção de picos.")
                print(f"  ⚠️ Vídeo {video_id} não possui trilha de áudio. Pulando detecção de picos.")

        except Exception as e:
            logging.error(f"Erro ao processar picos de áudio do vídeo {video_id}: {e}")
            print(f"  ❌ ERRO ao processar picos de áudio do vídeo {video_id}: {e}")
        finally:
            # Limpar arquivo de áudio temporário
            if os.path.exists(audio_output_path):
                os.remove(audio_output_path)
                logging.info(f"Arquivo temporário de áudio removido: {audio_output_path}")

    print("\n✅ SUBLAYER 3.2 (DETECÇÃO DE PICOS DE ÁUDIO) CONCLUÍDA")
    print(f"Todos os resultados de picos de áudio salvos em: {analise_audio_dir}")

# ============================================================================
# INSTRUÇÕES DE EXECUÇÃO NO GOOGLE COLAB:
# 1. Certifique-se de que a CÉLULA 1 (Setup Inicial) foi executada e instalou
#    as dependências `librosa`.
#    Adicione a seguinte linha na CÉLULA 1, se ainda não estiver lá:
#    `!pip install -q librosa`
# 2. Certifique-se de que a CÉLULA 2 (Configuração Inicial) foi executada e
#    definiu as variáveis globais `PASTA_TRABALHO` e `PASTA_VIDEOS`.
# 3. Certifique-se de que a CÉLULA 4 (Extração de Metadados) foi executada e
#    gerou o arquivo `metadados_completos.json`.
# 4. Copie e cole TODO o conteúdo desta célula em uma NOVA CÉLULA no seu
#    notebook Google Colab, logo APÓS a CÉLULA 4.2 (Transcrição de Áudio).
# 5. Execute esta nova célula.
# ============================================================================

# Bloco de execução principal para o Colab
if __name__ == '__main__':
    prereq_ok, videos_json_path = verificar_prerequisito_etapa4_audio_peaks()
    if prereq_ok:
        try:
            with open(videos_json_path, 'r', encoding='utf-8') as f:
                videos_data = json.load(f)
            processar_picos_audio_videos(videos_data)
        except Exception as e:
            print(f"❌ ERRO GERAL NA SUBLAYER 3.2 (DETECÇÃO DE PICOS DE ÁUDIO): {e}")
            logging.error(f"Erro geral na Sublayer 3.2 (Detecção de Picos de Áudio): {e}")






Progresso: 1/40 - Iniciando detecção de picos de áudio para o vídeo: vid_001 (entenda mais sobre a consciencia cristica.mp4)
  ✅ Detecção de picos de áudio concluída para vid_001. Encontrados 104 picos.
  Picos (primeiros 10): [0.11609977324263039, 0.18575963718820862, 0.3250793650793651, 0.3947392290249433, 0.5804988662131519, 0.6965986394557823, 0.8126984126984127, 0.905578231292517, 1.0216780045351475, 1.0913378684807256]...

Progresso: 2/40 - Iniciando detecção de picos de áudio para o vídeo: vid_002 (voce quer prosperar.mp4)
  ✅ Detecção de picos de áudio concluída para vid_002. Encontrados 215 picos.
  Picos (primeiros 10): [0.13931972789115646, 0.2089795918367347, 0.4179591836734694, 0.5804988662131519, 0.6965986394557823, 0.8359183673469388, 1.8808163265306121, 2.089795918367347, 2.2058956916099772, 2.345215419501134]...

Progresso: 3/40 - Iniciando detecção de picos de áudio para o vídeo: vid_003 (Por que é que muitos cristãos não prosperam financeiramente_ não acessam um

In [ ]:

# ============================================================================
# CÉLULA 4.4: SUBLAYER 3.3 - ANÁLISE DE SILÊNCIOS E PAUSAS ESTRATÉGICAS
# Esta célula deve ser executada APÓS a CÉLULA 4 (Extração de Metadados).
# ============================================================================

# Importar bibliotecas necessárias (certifique-se de que estejam instaladas na CÉLULA 1)
import os
import json
import logging
from moviepy.editor import VideoFileClip
from pydub import AudioSegment
from pydub.silence import detect_silence

def verificar_prerequisito_etapa4_silence():
    """Verifica se a extração de metadados foi executada com sucesso"""
    try:
        # Verifica se as variáveis globais estão definidas
        if not (
            'PASTA_TRABALHO' in globals() and
            'PASTA_VIDEOS' in globals()
        ):
            raise Exception("As variáveis globais PASTA_TRABALHO e PASTA_VIDEOS não estão definidas. Execute a CÉLULA 2 primeiro.")

        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        if not config['status_etapas']['metadados']:
            raise Exception("A etapa de metadados não foi concluída. Execute a CÉLULA 4 primeiro.")

        videos_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
        if not os.path.exists(videos_path):
            raise Exception("Metadados de vídeos não encontrados. Execute a CÉLULA 4 primeiro.")

        return True, videos_path
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO PARA ANÁLISE DE SILÊNCIOS: {e}")
        logging.error(f"Pré-requisito não atendido para análise de silêncios: {e}")
        return False, None

def analisar_silencios(audio_path):
    """Detecta silêncios em um arquivo de áudio e retorna seus intervalos."""
    try:
        audio = AudioSegment.from_wav(audio_path)
        # detect_silence retorna uma lista de [start_ms, end_ms]
        silence_ranges = detect_silence(audio,
                                        min_silence_len=700, # milissegundos mínimos para considerar silêncio
                                        silence_thresh=-40) # limite de decibéis para considerar silêncio

        silencios_info = []
        for start_ms, end_ms in silence_ranges:
            silencios_info.append({
                "inicio_segundos": round(start_ms / 1000.0, 2),
                "fim_segundos": round(end_ms / 1000.0, 2),
                "duracao_segundos": round((end_ms - start_ms) / 1000.0, 2)
            })
        return silencios_info
    except Exception as e:
        logging.error(f"Erro ao analisar silêncios em {audio_path}: {e}")
        return []

def processar_analise_silencios_videos(videos_info):
    """Processa a análise de silêncios para todos os vídeos e salva os resultados."""
    analise_audio_dir = os.path.join(PASTA_TRABALHO, "analise_audio")
    os.makedirs(analise_audio_dir, exist_ok=True)

    for i, video_info in enumerate(videos_info):
        video_id = video_info["id"]
        video_path = video_info["caminho_completo"]

        print(f"\nProgresso: {i+1}/{len(videos_info)} - Iniciando análise de silêncios para o vídeo: {video_id} ({video_info["nome_arquivo"]})")
        logging.info(f"Iniciando análise de silêncios para o vídeo: {video_id}")

        audio_output_path = os.path.join(PASTA_TRABALHO, "temp", f"{video_id}_audio_temp.wav")
        os.makedirs(os.path.dirname(audio_output_path), exist_ok=True)

        try:
            # Extrair áudio do vídeo
            clip = VideoFileClip(video_path)
            if clip.audio:
                clip.audio.write_audiofile(audio_output_path, verbose=False, logger=None)
                clip.close()
                logging.info(f"Áudio extraído para: {audio_output_path}")

                # Analisar silêncios
                silencios_encontrados = analisar_silencios(audio_output_path)

                output_json_path = os.path.join(analise_audio_dir, f"{video_id}_silencios.json")
                with open(output_json_path, 'w', encoding='utf-8') as f:
                    json.dump({"video_id": video_id, "silencios": silencios_encontrados}, f, indent=2, ensure_ascii=False)
                logging.info(f"Silêncios para {video_id} salvos em: {output_json_path}")
                print(f"  ✅ Análise de silêncios concluída para {video_id}. Encontrados {len(silencios_encontrados)} períodos de silêncio.")
                for silencio in silencios_encontrados[:3]: # Mostrar os 3 primeiros para resumo
                    print(f"    - Início: {silencio['inicio_segundos']}s, Fim: {silencio['fim_segundos']}s, Duração: {silencio['duracao_segundos']}s")
                if len(silencios_encontrados) > 3:
                    print("    ...")
            else:
                logging.warning(f"Vídeo {video_id} não possui trilha de áudio. Pulando análise de silêncios.")
                print(f"  ⚠️ Vídeo {video_id} não possui trilha de áudio. Pulando análise de silêncios.")

        except Exception as e:
            logging.error(f"Erro ao processar análise de silêncios do vídeo {video_id}: {e}")
            print(f"  ❌ ERRO ao processar análise de silêncios do vídeo {video_id}: {e}")
        finally:
            # Limpar arquivo de áudio temporário
            if os.path.exists(audio_output_path):
                os.remove(audio_output_path)
                logging.info(f"Arquivo temporário de áudio removido: {audio_output_path}")

    print("\n✅ SUBLAYER 3.3 (ANÁLISE DE SILÊNCIOS E PAUSAS) CONCLUÍDA")
    print(f"Todos os resultados de análise de silêncios salvos em: {analise_audio_dir}")

# ============================================================================
# INSTRUÇÕES DE EXECUÇÃO NO GOOGLE COLAB:
# 1. Certifique-se de que a CÉLULA 1 (Setup Inicial) foi executada e instalou
#    as dependências `pydub`.
#    Adicione a seguinte linha na CÉLULA 1, se ainda não estiver lá:
#    `!pip install -q pydub`
# 2. Certifique-se de que a CÉLULA 2 (Configuração Inicial) foi executada e
#    definiu as variáveis globais `PASTA_TRABALHO` e `PASTA_VIDEOS`.
# 3. Certifique-se de que a CÉLULA 4 (Extração de Metadados) foi executada e
#    gerou o arquivo `metadados_completos.json`.
# 4. Copie e cole TODO o conteúdo desta célula em uma NOVA CÉLULA no seu
#    notebook Google Colab, logo APÓS a CÉLULA 4.3 (Detecção de Picos de Áudio).
# 5. Execute esta nova célula.
# ============================================================================

# Bloco de execução principal para o Colab
if __name__ == '__main__':
    prereq_ok, videos_json_path = verificar_prerequisito_etapa4_silence()
    if prereq_ok:
        try:
            with open(videos_json_path, 'r', encoding='utf-8') as f:
                videos_data = json.load(f)
            processar_analise_silencios_videos(videos_data)
        except Exception as e:
            print(f"❌ ERRO GERAL NA SUBLAYER 3.3 (ANÁLISE DE SILÊNCIOS): {e}")
            logging.error(f"Erro geral na Sublayer 3.3 (Análise de Silêncios): {e}")






Progresso: 1/40 - Iniciando análise de silêncios para o vídeo: vid_001 (entenda mais sobre a consciencia cristica.mp4)
  ✅ Análise de silêncios concluída para vid_001. Encontrados 0 períodos de silêncio.

Progresso: 2/40 - Iniciando análise de silêncios para o vídeo: vid_002 (voce quer prosperar.mp4)
  ✅ Análise de silêncios concluída para vid_002. Encontrados 0 períodos de silêncio.

Progresso: 3/40 - Iniciando análise de silêncios para o vídeo: vid_003 (Por que é que muitos cristãos não prosperam financeiramente_ não acessam uma ruptura financeira.mp4)
  ✅ Análise de silêncios concluída para vid_003. Encontrados 0 períodos de silêncio.

Progresso: 4/40 - Iniciando análise de silêncios para o vídeo: vid_004 (como é que eu sei que algo que eu estou carregando é da parte de Deus Por dois motivos..mp4)
  ✅ Análise de silêncios concluída para vid_004. Encontrados 0 períodos de silêncio.

Progresso: 5/40 - Iniciando análise de silêncios para o vídeo: vid_005 (ja perdoei mas nao quer

In [ ]:

# ============================================================================
# CÉLULA 4.5: SUBLAYER 3.4 - ANÁLISE DE EFEITOS SONOROS E MÚSICA DE FUNDO
# Esta célula deve ser executada APÓS a CÉLULA 4 (Extração de Metadados).
# ============================================================================

# Importar bibliotecas necessárias (certifique-se de que estejam instaladas na CÉLULA 1)
import os
import json
import logging
import librosa
import numpy as np
from moviepy.editor import VideoFileClip

def verificar_prerequisito_etapa4_audio_effects():
    """Verifica se a extração de metadados foi executada com sucesso"""
    try:
        # Verifica se as variáveis globais estão definidas
        if not (
            'PASTA_TRABALHO' in globals() and
            'PASTA_VIDEOS' in globals()
        ):
            raise Exception("As variáveis globais PASTA_TRABALHO e PASTA_VIDEOS não estão definidas. Execute a CÉLULA 2 primeiro.")

        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        if not config['status_etapas']['metadados']:
            raise Exception("A etapa de metadados não foi concluída. Execute a CÉLULA 4 primeiro.")

        videos_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
        if not os.path.exists(videos_path):
            raise Exception("Metadados de vídeos não encontrados. Execute a CÉLULA 4 primeiro.")

        return True, videos_path
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO PARA ANÁLISE DE EFEITOS SONOROS E MÚSICA: {e}")
        logging.error(f"Pré-requisito não atendido para análise de efeitos sonoros e música: {e}")
        return False, None

def analisar_caracteristicas_audio(audio_path, sr_rate=22050):
    """Analisa características de áudio para inferir a presença de música/efeitos."""
    try:
        y, sr = librosa.load(audio_path, sr=sr_rate)

        # Calcular o RMS (Root Mean Square) de energia para detectar atividade sonora
        # Um RMS alto pode indicar música ou fala, enquanto um RMS baixo pode ser silêncio ou ruído de fundo.
        rms = librosa.feature.rms(y=y).flatten()

        # Calcular o Spectral Centroid para inferir o 'brilho' do som
        # Um centroid espectral mais alto pode indicar sons mais agudos, como efeitos sonoros ou vozes agudas.
        # Um centroid espectral mais baixo pode indicar sons mais graves, como música de fundo.
        cent = librosa.feature.spectral_centroid(y=y, sr=sr).flatten()

        # Calcular o Spectral Rolloff para inferir a forma do espectro
        # Pode ajudar a distinguir entre ruído e sons tonais.
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr).flatten()

        # Média das características para uma visão geral
        media_rms = np.mean(rms) if rms.size > 0 else 0
        media_cent = np.mean(cent) if cent.size > 0 else 0
        media_rolloff = np.mean(rolloff) if rolloff.size > 0 else 0

        # Inferência básica (pode ser aprimorada com machine learning)
        inferencia = ""
        if media_rms > 0.05: # Limiar arbitrário para atividade sonora
            if media_cent > 2500: # Limiar arbitrário para sons mais agudos
                inferencia = "Provável presença de efeitos sonoros ou fala ativa."
            elif media_cent > 1000: # Limiar arbitrário para sons médios
                inferencia = "Provável presença de música de fundo ou fala."
            else:
                inferencia = "Provável presença de música de fundo com tons graves."
        else:
            inferencia = "Baixa atividade sonora (silêncio ou ruído de fundo leve)."

        return {
            "media_rms": round(float(media_rms), 4),
            "media_spectral_centroid": round(float(media_cent), 2),
            "media_spectral_rolloff": round(float(media_rolloff), 2),
            "inferencia_basica": inferencia
        }
    except Exception as e:
        logging.error(f"Erro ao analisar características de áudio em {audio_path}: {e}")
        return {"erro": str(e)}

def processar_analise_audio_efeitos_videos(videos_info):
    """Processa a análise de efeitos sonoros e música de fundo para todos os vídeos e salva os resultados."""
    analise_audio_dir = os.path.join(PASTA_TRABALHO, "analise_audio")
    os.makedirs(analise_audio_dir, exist_ok=True)

    for i, video_info in enumerate(videos_info):
        video_id = video_info["id"]
        video_path = video_info["caminho_completo"]

        print(f"\nProgresso: {i+1}/{len(videos_info)} - Iniciando análise de efeitos sonoros e música para o vídeo: {video_id} ({video_info["nome_arquivo"]})")
        logging.info(f"Iniciando análise de efeitos sonoros e música para o vídeo: {video_id}")

        audio_output_path = os.path.join(PASTA_TRABALHO, "temp", f"{video_id}_audio_temp.wav")
        os.makedirs(os.path.dirname(audio_output_path), exist_ok=True)

        try:
            # Extrair áudio do vídeo
            clip = VideoFileClip(video_path)
            if clip.audio:
                clip.audio.write_audiofile(audio_output_path, verbose=False, logger=None)
                clip.close()
                logging.info(f"Áudio extraído para: {audio_output_path}")

                # Analisar características de áudio
                caracteristicas_audio = analisar_caracteristicas_audio(audio_output_path)

                output_json_path = os.path.join(analise_audio_dir, f"{video_id}_caracteristicas_audio.json")
                with open(output_json_path, 'w', encoding='utf-8') as f:
                    json.dump({"video_id": video_id, "caracteristicas_audio": caracteristicas_audio}, f, indent=2, ensure_ascii=False)
                logging.info(f"Características de áudio para {video_id} salvas em: {output_json_path}")
                print(f"  ✅ Análise de características de áudio concluída para {video_id}.")
                print(f"  Inferência: {caracteristicas_audio.get('inferencia_basica', 'N/A')}")
                print(f"  Média RMS: {caracteristicas_audio.get('media_rms', 'N/A')}, Centroid: {caracteristicas_audio.get('media_spectral_centroid', 'N/A')}")
            else:
                logging.warning(f"Vídeo {video_id} não possui trilha de áudio. Pulando análise de efeitos sonoros e música.")
                print(f"  ⚠️ Vídeo {video_id} não possui trilha de áudio. Pulando análise de efeitos sonoros e música.")

        except Exception as e:
            logging.error(f"Erro ao processar análise de efeitos sonoros e música do vídeo {video_id}: {e}")
            print(f"  ❌ ERRO ao processar análise de efeitos sonoros e música do vídeo {video_id}: {e}")
        finally:
            # Limpar arquivo de áudio temporário
            if os.path.exists(audio_output_path):
                os.remove(audio_output_path)
                logging.info(f"Arquivo temporário de áudio removido: {audio_output_path}")

    print("\n✅ SUBLAYER 3.4 (ANÁLISE DE EFEITOS SONOROS E MÚSICA DE FUNDO) CONCLUÍDA")
    print(f"Todos os resultados de análise de áudio salvos em: {analise_audio_dir}")

# ============================================================================
# INSTRUÇÕES DE EXECUÇÃO NO GOOGLE COLAB:
# 1. Certifique-se de que a CÉLULA 1 (Setup Inicial) foi executada e instalou
#    as dependências `librosa`.
#    Adicione a seguinte linha na CÉLULA 1, se ainda não estiver lá:
#    `!pip install -q librosa`
# 2. Certifique-se de que a CÉLULA 2 (Configuração Inicial) foi executada e
#    definiu as variáveis globais `PASTA_TRABALHO` e `PASTA_VIDEOS`.
# 3. Certifique-se de que a CÉLULA 4 (Extração de Metadados) foi executada e
#    gerou o arquivo `metadados_completos.json`.
# 4. Copie e cole TODO o conteúdo desta célula em uma NOVA CÉLULA no seu
#    notebook Google Colab, logo APÓS a CÉLULA 4.4 (Análise de Silêncios).
# 5. Execute esta nova célula.
# ============================================================================

# Bloco de execução principal para o Colab
if __name__ == '__main__':
    prereq_ok, videos_json_path = verificar_prerequisito_etapa4_audio_effects()
    if prereq_ok:
        try:
            with open(videos_json_path, 'r', encoding='utf-8') as f:
                videos_data = json.load(f)
            processar_analise_audio_efeitos_videos(videos_data)
        except Exception as e:
            print(f"❌ ERRO GERAL NA SUBLAYER 3.4 (ANÁLISE DE EFEITOS SONOROS E MÚSICA): {e}")
            logging.error(f"Erro geral na Sublayer 3.4 (Análise de Efeitos Sonoros e Música): {e}")




Progresso: 1/40 - Iniciando análise de efeitos sonoros e música para o vídeo: vid_001 (entenda mais sobre a consciencia cristica.mp4)
  ✅ Análise de características de áudio concluída para vid_001.
  Inferência: Provável presença de música de fundo ou fala.
  Média RMS: 0.1315, Centroid: 1788.14

Progresso: 2/40 - Iniciando análise de efeitos sonoros e música para o vídeo: vid_002 (voce quer prosperar.mp4)
  ✅ Análise de características de áudio concluída para vid_002.
  Inferência: Provável presença de música de fundo ou fala.
  Média RMS: 0.1241, Centroid: 1866.16

Progresso: 3/40 - Iniciando análise de efeitos sonoros e música para o vídeo: vid_003 (Por que é que muitos cristãos não prosperam financeiramente_ não acessam uma ruptura financeira.mp4)
  ✅ Análise de características de áudio concluída para vid_003.
  Inferência: Provável presença de música de fundo ou fala.
  Média RMS: 0.122, Centroid: 2326.33

Progresso: 4/40 - Iniciando análise de efeitos sonoros e música para o

In [ ]:

# ============================================================================
# CÉLULA 4.6: SUBLAYER 3.5 - ANÁLISE DE BATIMENTO CARDÍACO (BPM)
# Esta célula deve ser executada APÓS a CÉLULA 4 (Extração de Metadados).
# ============================================================================

# Importar bibliotecas necessárias (certifique-se de que estejam instaladas na CÉLULA 1)
import os
import json
import logging
import librosa
import numpy as np
from moviepy.editor import VideoFileClip

def verificar_prerequisito_etapa4_bpm():
    """Verifica se a extração de metadados foi executada com sucesso"""
    try:
        # Verifica se as variáveis globais estão definidas
        if not (
            'PASTA_TRABALHO' in globals() and
            'PASTA_VIDEOS' in globals()
        ):
            raise Exception("As variáveis globais PASTA_TRABALHO e PASTA_VIDEOS não estão definidas. Execute a CÉLULA 2 primeiro.")

        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        if not config['status_etapas']['metadados']:
            raise Exception("A etapa de metadados não foi concluída. Execute a CÉLULA 4 primeiro.")

        videos_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
        if not os.path.exists(videos_path):
            raise Exception("Metadados de vídeos não encontrados. Execute a CÉLULA 4 primeiro.")

        return True, videos_path
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO PARA ANÁLISE DE BPM: {e}")
        logging.error(f"Pré-requisito não atendido para análise de BPM: {e}")
        return False, None

def detectar_bpm(audio_path, sr_rate=22050):
    """Detecta o BPM (Batidas Por Minuto) de um arquivo de áudio."""
    try:
        print(f"🔊 Carregando áudio: {os.path.basename(audio_path)}")
        y, sr = librosa.load(audio_path, sr=sr_rate)

        if len(y) == 0:
            print("⚠️ Arquivo de áudio vazio")
            return None

        # MÉTODO CORRIGIDO - sem onset_env
        print("🎵 Detectando BPM...")
        tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)

        # Validar resultado
        if tempo > 0:
            bpm = round(float(tempo), 2)
            print(f"✅ BPM detectado: {bpm}")
            return bpm
        else:
            print("⚠️ BPM não pôde ser determinado")
            return None

    except Exception as e:
        print(f"❌ Erro ao detectar BPM: {e}")
        logging.error(f"Erro ao detectar BPM em {audio_path}: {e}")
        return None

def processar_bpm_videos(videos_info):
    """Processa a detecção de BPM para todos os vídeos e salva os resultados."""
    analise_audio_dir = os.path.join(PASTA_TRABALHO, "analise_audio")
    os.makedirs(analise_audio_dir, exist_ok=True)

    for i, video_info in enumerate(videos_info):
        video_id = video_info["id"]
        video_path = video_info["caminho_completo"]

        print(f"\nProgresso: {i+1}/{len(videos_info)} - Iniciando análise de BPM para o vídeo: {video_id} ({video_info["nome_arquivo"]})")
        logging.info(f"Iniciando análise de BPM para o vídeo: {video_id}")

        audio_output_path = os.path.join(PASTA_TRABALHO, "temp", f"{video_id}_audio_temp.wav")
        os.makedirs(os.path.dirname(audio_output_path), exist_ok=True)

        try:
            # Extrair áudio do vídeo
            clip = VideoFileClip(video_path)
            if clip.audio:
                clip.audio.write_audiofile(audio_output_path, verbose=False, logger=None)
                clip.close()
                logging.info(f"Áudio extraído para: {audio_output_path}")

                # Detectar BPM
                bpm = detectar_bpm(audio_output_path)

                output_json_path = os.path.join(analise_audio_dir, f"{video_id}_bpm.json")
                with open(output_json_path, 'w', encoding='utf-8') as f:
                    json.dump({"video_id": video_id, "bpm": bpm}, f, indent=2, ensure_ascii=False)
                logging.info(f"BPM para {video_id} salvo em: {output_json_path}")
                print(f"  ✅ Análise de BPM concluída para {video_id}. BPM detectado: {bpm}")
            else:
                logging.warning(f"Vídeo {video_id} não possui trilha de áudio. Pulando análise de BPM.")
                print(f"  ⚠️ Vídeo {video_id} não possui trilha de áudio. Pulando análise de BPM.")

        except Exception as e:
            logging.error(f"Erro ao processar análise de BPM do vídeo {video_id}: {e}")
            print(f"  ❌ ERRO ao processar análise de BPM do vídeo {video_id}: {e}")
        finally:
            # Limpar arquivo de áudio temporário
            if os.path.exists(audio_output_path):
                os.remove(audio_output_path)
                logging.info(f"Arquivo temporário de áudio removido: {audio_output_path}")

    print("\n✅ SUBLAYER 3.5 (ANÁLISE DE BPM) CONCLUÍDA")
    print(f"Todos os resultados de BPM salvos em: {analise_audio_dir}")

# ============================================================================
# INSTRUÇÕES DE EXECUÇÃO NO GOOGLE COLAB:
# 1. Certifique-se de que a CÉLULA 1 (Setup Inicial) foi executada e instalou
#    as dependências `librosa`.
#    Adicione a seguinte linha na CÉLULA 1, se ainda não estiver lá:
#    `!pip install -q librosa`
# 2. Certifique-se de que a CÉLULA 2 (Configuração Inicial) foi executada e
#    definiu as variáveis globais `PASTA_TRABALHO` e `PASTA_VIDEOS`.
# 3. Certifique-se de que a CÉLULA 4 (Extração de Metadados) foi executada e
#    gerou o arquivo `metadados_completos.json`.
# 4. Copie e cole TODO o conteúdo desta célula em uma NOVA CÉLULA no seu
#    notebook Google Colab, logo APÓS a CÉLULA 4.5 (Análise de Efeitos Sonoros e Música).
# 5. Execute esta nova célula.
# ============================================================================

# Bloco de execução principal para o Colab
if __name__ == '__main__':
    prereq_ok, videos_json_path = verificar_prerequisito_etapa4_bpm()
    if prereq_ok:
        try:
            with open(videos_json_path, 'r', encoding='utf-8') as f:
                videos_data = json.load(f)
            processar_bpm_videos(videos_data)
        except Exception as e:
            print(f"❌ ERRO GERAL NA SUBLAYER 3.5 (ANÁLISE DE BPM): {e}")
            logging.error(f"Erro geral na Sublayer 3.5 (Análise de BPM): {e}")






Progresso: 1/40 - Iniciando análise de BPM para o vídeo: vid_001 (entenda mais sobre a consciencia cristica.mp4)
🔊 Carregando áudio: vid_001_audio_temp.wav
🎵 Detectando BPM...
✅ BPM detectado: 95.7
  ✅ Análise de BPM concluída para vid_001. BPM detectado: 95.7

Progresso: 2/40 - Iniciando análise de BPM para o vídeo: vid_002 (voce quer prosperar.mp4)
🔊 Carregando áudio: vid_002_audio_temp.wav
🎵 Detectando BPM...
✅ BPM detectado: 136.0
  ✅ Análise de BPM concluída para vid_002. BPM detectado: 136.0

Progresso: 3/40 - Iniciando análise de BPM para o vídeo: vid_003 (Por que é que muitos cristãos não prosperam financeiramente_ não acessam uma ruptura financeira.mp4)
🔊 Carregando áudio: vid_003_audio_temp.wav
🎵 Detectando BPM...
✅ BPM detectado: 107.67
  ✅ Análise de BPM concluída para vid_003. BPM detectado: 107.67

Progresso: 4/40 - Iniciando análise de BPM para o vídeo: vid_004 (como é que eu sei que algo que eu estou carregando é da parte de Deus Por dois motivos..mp4)
🔊 Carregan

In [ ]:

# ============================================================================
# CÉLULA 5.1: SUBLAYER 4.1 - DETECÇÃO DE EMOÇÕES
# Esta célula deve ser executada APÓS as CÉLULA 4 (Metadados), CÉLULA 4.1 (OCR)
# e CÉLULA 4.2 (Transcrição de Áudio).
# ============================================================================

# Importar bibliotecas necessárias
import os
import json
import logging

def verificar_prerequisito_etapa_emocoes():
    """Verifica se as etapas anteriores (metadados, OCR, transcrição de áudio) foram executadas com sucesso."""
    try:
        if not (
            'PASTA_TRABALHO' in globals() and
            'PASTA_VIDEOS' in globals()
        ):
            raise Exception("As variáveis globais PASTA_TRABALHO e PASTA_VIDEOS não estão definidas. Execute a CÉLULA 2 primeiro.")

        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        if not config['status_etapas']['metadados']:
            raise Exception("A etapa de metadados não foi concluída. Execute a CÉLULA 4 primeiro.")

        videos_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
        if not os.path.exists(videos_path):
            raise Exception("Metadados de vídeos não encontrados. Execute a CÉLULA 4 primeiro.")

        # Verificar se os resultados de OCR e Transcrição de Áudio existem
        analise_texto_dir = os.path.join(PASTA_TRABALHO, "analise_texto")
        analise_audio_transcricoes_dir = os.path.join(PASTA_TRABALHO, "analise_audio", "transcricoes")

        if not os.path.exists(analise_texto_dir) or not os.path.exists(analise_audio_transcricoes_dir):
            logging.warning("Diretórios de OCR ou Transcrição de Áudio não encontrados. Certifique-se de que as CÉLULAS 4.1 e 4.2 foram executadas.")
            # Não é um erro fatal, pois pode haver vídeos sem áudio ou texto

        return True, videos_path
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO PARA DETECÇÃO DE EMOÇÕES: {e}")
        logging.error(f"Pré-requisito não atendido para detecção de emoções: {e}")
        return False, None

def detectar_emocoes(texto):
    """Detecta emoções em um texto usando uma abordagem baseada em palavras-chave."""
    emocoes_detectadas = {
        "curiosidade": 0,
        "urgencia": 0,
        "validacao": 0,
        "aspiracao": 0,
        "medo_de_perder": 0,
        "surpresa": 0,
        "humor": 0
    }

    # Listas de palavras-chave para cada emoção (em português)
    palavras_chave = {
        "curiosidade": ["segredo", "descobrir", "verdade", "ninguém sabe", "o que acontece se", "revelado", "mistério"],
        "urgencia": ["agora", "rápido", "antes que seja tarde", "não perca", "última chance", "imediatamente", "hoje"],
        "validacao": ["você não está sozinho", "eu sei como é", "isso é para quem", "familiar", "entendo"],
        "aspiracao": ["sucesso", "transformar", "melhorar", "alcançar", "sonho", "potencial", "futuro"],
        "medo_de_perder": ["perder", "erro", "não faça", "atrasado", "arrepender", "evite"],
        "surpresa": ["chocante", "inacreditável", "incrível", "inesperado", "uau"],
        "humor": ["engraçado", "risada", "piada", "divertido", "haha"]
    }

    texto_lower = texto.lower()

    for emocao, palavras in palavras_chave.items():
        for palavra in palavras:
            if palavra in texto_lower:
                emocoes_detectadas[emocao] += 1

    # Retorna as emoções com contagem maior que zero
    return {e: c for e, c in emocoes_detectadas.items() if c > 0}


def processar_deteccao_emocoes_videos(videos_info):
    """Processa a detecção de emoções para todos os vídeos e salva os resultados."""
    analise_psicologica_dir = os.path.join(PASTA_TRABALHO, "analise_psicologica")
    os.makedirs(analise_psicologica_dir, exist_ok=True)

    analise_texto_dir = os.path.join(PASTA_TRABALHO, "analise_texto")
    analise_audio_transcricoes_dir = os.path.join(PASTA_TRABALHO, "analise_audio", "transcricoes")

    for i, video_info in enumerate(videos_info):
        video_id = video_info["id"]

        print(f"\nProgresso: {i+1}/{len(videos_info)} - Iniciando detecção de emoções para o vídeo: {video_id} ({video_info["nome_arquivo"]})")
        logging.info(f"Iniciando detecção de emoções para o vídeo: {video_id}")

        texto_combinado = ""

        # Carregar texto do OCR
        ocr_path = os.path.join(analise_texto_dir, f"{video_id}_ocr_results.json")
        if os.path.exists(ocr_path):
            try:
                with open(ocr_path, 'r', encoding='utf-8') as f:
                    ocr_data = json.load(f)
                for item in ocr_data:
                    texto_combinado += item.get("texto_extraido", "") + " "
                logging.info(f"Texto OCR carregado para {video_id}.")
            except Exception as e:
                logging.error(f"Erro ao carregar OCR para {video_id}: {e}")

        # Carregar texto da transcrição de áudio
        transcricao_audio_path = os.path.join(analise_audio_transcricoes_dir, f"{video_id}_transcricao.txt")
        if os.path.exists(transcricao_audio_path):
            try:
                with open(transcricao_audio_path, 'r', encoding='utf-8') as f:
                    texto_combinado += f.read() + " "
                logging.info(f"Transcrição de áudio carregada para {video_id}.")
            except Exception as e:
                logging.error(f"Erro ao carregar transcrição de áudio para {video_id}: {e}")

        emocoes_encontradas = {}
        if texto_combinado.strip():
            emocoes_encontradas = detectar_emocoes(texto_combinado)
            logging.info(f"Emoções detectadas para {video_id}: {emocoes_encontradas}")
            print(f"  ✅ Detecção de emoções concluída para {video_id}.")
            if emocoes_encontradas:
                print(f"  Emoções detectadas: {', '.join([f'{e} ({c})' for e, c in emocoes_encontradas.items()])}")
            else:
                print("  Nenhuma emoção específica detectada com as palavras-chave atuais.")
        else:
            logging.warning(f"Nenhum texto disponível (OCR ou Transcrição) para o vídeo {video_id}. Pulando detecção de emoções.")
            print(f"  ⚠️ Nenhum texto disponível para o vídeo {video_id}. Pulando detecção de emoções.")

        output_json_path = os.path.join(analise_psicologica_dir, f"{video_id}_emocoes_detectadas.json")
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump({"video_id": video_id, "emocoes": emocoes_encontradas}, f, indent=2, ensure_ascii=False)
        logging.info(f"Resultados de emoções para {video_id} salvos em: {output_json_path}")

    print("\n✅ SUBLAYER 4.1 (DETECÇÃO DE EMOÇÕES) CONCLUÍDA")
    print(f"Todos os resultados de detecção de emoções salvos em: {analise_psicologica_dir}")

# ============================================================================
# INSTRUÇÕES DE EXECUÇÃO NO GOOGLE COLAB:
# 1. Certifique-se de que a CÉLULA 1 (Setup Inicial) foi executada.
# 2. Certifique-se de que a CÉLULA 2 (Configuração Inicial) foi executada e
#    definiu as variáveis globais `PASTA_TRABALHO` e `PASTA_VIDEOS`.
# 3. Certifique-se de que a CÉLULA 4 (Extração de Metadados) foi executada e
#    gerou o arquivo `metadados_completos.json`.
# 4. Certifique-se de que a CÉLULA 4.1 (OCR) e a CÉLULA 4.2 (Transcrição de Áudio)
#    foram executadas e geraram seus respectivos arquivos de resultados.
# 5. Copie e cole TODO o conteúdo desta célula em uma NOVA CÉLULA no seu
#    notebook Google Colab, logo APÓS a CÉLULA 4.6 (Análise de BPM).
# 6. Execute esta nova célula.
# ============================================================================

# Bloco de execução principal para o Colab
if __name__ == '__main__':
    prereq_ok, videos_json_path = verificar_prerequisito_etapa_emocoes()
    if prereq_ok:
        try:
            with open(videos_json_path, 'r', encoding='utf-8') as f:
                videos_data = json.load(f)
            processar_deteccao_emocoes_videos(videos_data)
        except Exception as e:
            print(f"❌ ERRO GERAL NA SUBLAYER 4.1 (DETECÇÃO DE EMOÇÕES): {e}")
            logging.error(f"Erro geral na Sublayer 4.1 (Detecção de Emoções): {e}")






Progresso: 1/40 - Iniciando detecção de emoções para o vídeo: vid_001 (entenda mais sobre a consciencia cristica.mp4)
  ✅ Detecção de emoções concluída para vid_001.
  Nenhuma emoção específica detectada com as palavras-chave atuais.

Progresso: 2/40 - Iniciando detecção de emoções para o vídeo: vid_002 (voce quer prosperar.mp4)
  ✅ Detecção de emoções concluída para vid_002.
  Emoções detectadas: urgencia (1)

Progresso: 3/40 - Iniciando detecção de emoções para o vídeo: vid_003 (Por que é que muitos cristãos não prosperam financeiramente_ não acessam uma ruptura financeira.mp4)
  ✅ Detecção de emoções concluída para vid_003.
  Emoções detectadas: curiosidade (1)

Progresso: 4/40 - Iniciando detecção de emoções para o vídeo: vid_004 (como é que eu sei que algo que eu estou carregando é da parte de Deus Por dois motivos..mp4)
  ✅ Detecção de emoções concluída para vid_004.
  Emoções detectadas: aspiracao (1)

Progresso: 5/40 - Iniciando detecção de emoções para o vídeo: vid_005 

In [ ]:

# ============================================================================
# CÉLULA 5.2: SUBLAYER 4.2 - IDENTIFICAÇÃO DE "GANCHOS DE DOPAMINA"
# Esta célula deve ser executada APÓS as CÉLULA 4 (Metadados), CÉLULA 4.1 (OCR)
# e CÉLULA 4.2 (Transcrição de Áudio).
# ============================================================================

# Importar bibliotecas necessárias
import os
import json
import logging

def verificar_prerequisito_etapa_ganchos():
    """Verifica se as etapas anteriores (metadados, OCR, transcrição de áudio) foram executadas com sucesso."""
    try:
        if not (
            'PASTA_TRABALHO' in globals() and
            'PASTA_VIDEOS' in globals()
        ):
            raise Exception("As variáveis globais PASTA_TRABALHO e PASTA_VIDEOS não estão definidas. Execute a CÉLULA 2 primeiro.")

        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        if not config['status_etapas']['metadados']:
            raise Exception("A etapa de metadados não foi concluída. Execute a CÉLULA 4 primeiro.")

        videos_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
        if not os.path.exists(videos_path):
            raise Exception("Metadados de vídeos não encontrados. Execute a CÉLULA 4 primeiro.")

        # Verificar se os resultados de OCR e Transcrição de Áudio existem
        analise_texto_dir = os.path.join(PASTA_TRABALHO, "analise_texto")
        analise_audio_transcricoes_dir = os.path.join(PASTA_TRABALHO, "analise_audio", "transcricoes")

        if not os.path.exists(analise_texto_dir) or not os.path.exists(analise_audio_transcricoes_dir):
            logging.warning("Diretórios de OCR ou Transcrição de Áudio não encontrados. Certifique-se de que as CÉLULAS 4.1 e 4.2 foram executadas.")
            # Não é um erro fatal, pois pode haver vídeos sem áudio ou texto

        return True, videos_path
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO PARA IDENTIFICAÇÃO DE GANCHOS DE DOPAMINA: {e}")
        logging.error(f"Pré-requisito não atendido para identificação de ganchos de dopamina: {e}")
        return False, None

def identificar_ganchos_dopamina(texto):
    """Identifica ganchos de dopamina em um texto baseado em categorias e frases."""
    ganchos_encontrados = []

    # Dicionário de ganchos de dopamina baseado no PDF fornecido
    # As frases foram simplificadas para facilitar a detecção por substring.
    # Para maior precisão, seria necessário NLP mais avançado (tokenização, lematização, etc.)
    ganchos_dopamina_categorias = {
        "Curiosidade / Segredos": [
            "ninguém fala sobre isso mas", "a verdade sobre", "eles não querem que você saiba",
            "o que acontece se você tentar isso", "eu queria que alguém tivesse me dito isso antes",
            "isso parece ilegal de saber", "escondido à vista de todos", "você nunca mais vai olhar para",
            "o que eu descobri depois de", "a parte que eles nunca te mostram", "eu tentei isso para que você não precise",
            "a fórmula secreta por trás de", "eles nunca vão admitir isso", "a coisa #1 que ninguém ensina sobre",
            "eu finalmente descobri por que não funciona"
        ],
        "Medo de Perder (FOMO)": [
            "se você não está fazendo isso, está perdendo dinheiro", "você vai se arrepender de pular isso",
            "todo mundo está fazendo isso, menos você", "não cometa o mesmo erro que eu",
            "isso pode te economizar anos", "99% das pessoas ignoram esta etapa", "pare de rolar ou você vai perder isso",
            "esta oportunidade não vai durar", "o maior erro que iniciantes cometem",
            "se você não sabe disso, já está atrasado", "esta é sua última chance de",
            "perder isso está te custando", "se você pular isso, vai se arrepender depois",
            "não comece antes de assistir isso", "você vai se agradecer depois por fazer isso agora"
        ],
        "Validação / Identificação": [
            "se você já se sentiu assim, isso é para você", "eu sei exatamente como é quando",
            "isso é para quem está cansado de", "me diga se isso soa familiar",
            "eu costumava pensar que era o único que", "se você está lutando com, você não está sozinho",
            "já se perguntou por que você sempre se sente", "isso costumava ser eu todos os dias",
            "se isso acontece com você, você não é louco", "você tem trabalhado tanto, mas ainda se sente preso?",
            "apostou que ninguém nunca te disse isso", "se você está assistindo isso, não é por acaso",
            "este é o sinal que você estava esperando", "se você continua ouvindo 'apenas seja paciente'",
            "para todos que já foram subestimados"
        ],
        "Urgência / Comando": [
            "faça isso antes que seja tarde demais", "pare o que está fazendo e assista isso",
            "você precisa ouvir isso agora", "não espere mais um dia para começar",
            "tente isso imediatamente", "ouça isso com atenção", "assista isso duas vezes se te tocar",
            "anote isso antes que você esqueça", "salve este vídeo, você vai precisar dele depois",
            "nos próximos 10 segundos, você verá por que isso importa", "não durma nesta oportunidade",
            "o relógio está correndo, comece hoje", "eu queria ter começado isso antes",
            "faça isso hoje, agradeça a si mesmo amanhã", "pare de esperar, comece a se mover"
        ],
        "Transformação / Desejo": [
            "isso mudou minha vida completamente", "de quebrado para em 6 meses",
            "como eu fui de para em 30 dias", "a maneira mais rápida de sem",
            "se eu consigo, você também consegue", "o sistema que eu usei para ir de zero a",
            "quer? faça isso", "é assim que pessoas comuns se tornam",
            "os passos exatos que eu usei para alcançar", "se você começar isso hoje, seu eu futuro vai te agradecer",
            "este é o projeto que eu queria ter", "é assim que se consegue sem experiência",
            "o caminho de para explicado em 20 segundos", "é assim que a verdadeira liberdade se parece",
            "uma decisão mudou tudo para mim"
        ],
        "Choque / Contrário": [
            "você foi enganado sobre", "esqueça tudo o que você sabe sobre",
            "o pior conselho que já me deram foi", "pare de acreditar neste mito"
        ]
    }

    texto_lower = texto.lower()

    for categoria, frases in ganchos_dopamina_categorias.items():
        for frase in frases:
            if frase in texto_lower:
                ganchos_encontrados.append({
                    "categoria": categoria,
                    "frase_detectada": frase
                })
    return ganchos_encontrados

def processar_ganchos_dopamina_videos(videos_info):
    """Processa a identificação de ganchos de dopamina para todos os vídeos e salva os resultados."""
    analise_psicologica_dir = os.path.join(PASTA_TRABALHO, "analise_psicologica")
    os.makedirs(analise_psicologica_dir, exist_ok=True)

    analise_texto_dir = os.path.join(PASTA_TRABALHO, "analise_texto")
    analise_audio_transcricoes_dir = os.path.join(PASTA_TRABALHO, "analise_audio", "transcricoes")

    for i, video_info in enumerate(videos_info):
        video_id = video_info["id"]

        print(f"\nProgresso: {i+1}/{len(videos_info)} - Iniciando identificação de ganchos de dopamina para o vídeo: {video_id} ({video_info["nome_arquivo"]})")
        logging.info(f"Iniciando identificação de ganchos de dopamina para o vídeo: {video_id}")

        texto_combinado = ""

        # Carregar texto do OCR
        ocr_path = os.path.join(analise_texto_dir, f"{video_id}_ocr_results.json")
        if os.path.exists(ocr_path):
            try:
                with open(ocr_path, 'r', encoding='utf-8') as f:
                    ocr_data = json.load(f)
                for item in ocr_data:
                    texto_combinado += item.get("texto_extraido", "") + " "
                logging.info(f"Texto OCR carregado para {video_id}.")
            except Exception as e:
                logging.error(f"Erro ao carregar OCR para {video_id}: {e}")

        # Carregar texto da transcrição de áudio
        transcricao_audio_path = os.path.join(analise_audio_transcricoes_dir, f"{video_id}_transcricao.txt")
        if os.path.exists(transcricao_audio_path):
            try:
                with open(transcricao_audio_path, 'r', encoding='utf-8') as f:
                    texto_combinado += f.read() + " "
                logging.info(f"Transcrição de áudio carregada para {video_id}.")
            except Exception as e:
                logging.error(f"Erro ao carregar transcrição de áudio para {video_id}: {e}")

        ganchos_encontrados = []
        if texto_combinado.strip():
            ganchos_encontrados = identificar_ganchos_dopamina(texto_combinado)
            logging.info(f"Ganchos de dopamina detectados para {video_id}: {ganchos_encontrados}")
            print(f"  ✅ Identificação de ganchos de dopamina concluída para {video_id}.")
            if ganchos_encontrados:
                print(f"  Ganchos detectados: {[g['frase_detectada'] for g in ganchos_encontrados]}")
            else:
                print("  Nenhum gancho de dopamina específico detectado com as frases-chave atuais.")
        else:
            logging.warning(f"Nenhum texto disponível (OCR ou Transcrição) para o vídeo {video_id}. Pulando identificação de ganchos de dopamina.")
            print(f"  ⚠️ Nenhum texto disponível para o vídeo {video_id}. Pulando identificação de ganchos de dopamina.")

        output_json_path = os.path.join(analise_psicologica_dir, f"{video_id}_ganchos_dopamina.json")
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump({"video_id": video_id, "ganchos_dopamina": ganchos_encontrados}, f, indent=2, ensure_ascii=False)
        logging.info(f"Resultados de ganchos de dopamina para {video_id} salvos em: {output_json_path}")

    print("\n✅ SUBLAYER 4.2 (IDENTIFICAÇÃO DE GANCHOS DE DOPAMINA) CONCLUÍDA")
    print(f"Todos os resultados de ganchos de dopamina salvos em: {analise_psicologica_dir}")

# ============================================================================
# INSTRUÇÕES DE EXECUÇÃO NO GOOGLE COLAB:
# 1. Certifique-se de que a CÉLULA 1 (Setup Inicial) foi executada.
# 2. Certifique-se de que a CÉLULA 2 (Configuração Inicial) foi executada e
#    definiu as variáveis globais `PASTA_TRABALHO` e `PASTA_VIDEOS`.
# 3. Certifique-se de que a CÉLULA 4 (Extração de Metadados) foi executada e
#    gerou o arquivo `metadados_completos.json`.
# 4. Certifique-se de que a CÉLULA 4.1 (OCR) e a CÉLULA 4.2 (Transcrição de Áudio)
#    foram executadas e geraram seus respectivos arquivos de resultados.
# 5. Copie e cole TODO o conteúdo desta célula em uma NOVA CÉLULA no seu
#    notebook Google Colab, logo APÓS a CÉLULA 5.1 (Detecção de Emoções).
# 6. Execute esta nova célula.
# ============================================================================

# Bloco de execução principal para o Colab
if __name__ == '__main__':
    prereq_ok, videos_json_path = verificar_prerequisito_etapa_ganchos()
    if prereq_ok:
        try:
            with open(videos_json_path, 'r', encoding='utf-8') as f:
                videos_data = json.load(f)
            processar_ganchos_dopamina_videos(videos_data)
        except Exception as e:
            print(f"❌ ERRO GERAL NA SUBLAYER 4.2 (IDENTIFICAÇÃO DE GANCHOS DE DOPAMINA): {e}")
            logging.error(f"Erro geral na Sublayer 4.2 (Identificação de Ganchos de Dopamina): {e}")






Progresso: 1/40 - Iniciando identificação de ganchos de dopamina para o vídeo: vid_001 (entenda mais sobre a consciencia cristica.mp4)
  ✅ Identificação de ganchos de dopamina concluída para vid_001.
  Nenhum gancho de dopamina específico detectado com as frases-chave atuais.

Progresso: 2/40 - Iniciando identificação de ganchos de dopamina para o vídeo: vid_002 (voce quer prosperar.mp4)
  ✅ Identificação de ganchos de dopamina concluída para vid_002.
  Nenhum gancho de dopamina específico detectado com as frases-chave atuais.

Progresso: 3/40 - Iniciando identificação de ganchos de dopamina para o vídeo: vid_003 (Por que é que muitos cristãos não prosperam financeiramente_ não acessam uma ruptura financeira.mp4)
  ✅ Identificação de ganchos de dopamina concluída para vid_003.
  Nenhum gancho de dopamina específico detectado com as frases-chave atuais.

Progresso: 4/40 - Iniciando identificação de ganchos de dopamina para o vídeo: vid_004 (como é que eu sei que algo que eu estou 

In [ ]:

# ============================================================================
# CÉLULA 4.7: SUBLAYER 1.1 - TIMING DE CORTES E TRANSIÇÕES
# Esta célula deve ser executada APÓS a CÉLULA 4 (Extração de Metadados).
# ============================================================================

# Importar bibliotecas necessárias (certifique-se de que estejam instaladas na CÉLULA 1)
import os
import json
import logging
import cv2
import numpy as np

def verificar_prerequisito_etapa_cortes():
    """Verifica se a extração de metadados foi executada com sucesso."""
    try:
        if not (
            'PASTA_TRABALHO' in globals() and
            'PASTA_VIDEOS' in globals()
        ):
            raise Exception("As variáveis globais PASTA_TRABALHO e PASTA_VIDEOS não estão definidas. Execute a CÉLULA 2 primeiro.")

        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        if not config['status_etapas']['metadados']:
            raise Exception("A etapa de metadados não foi concluída. Execute a CÉLULA 4 primeiro.")

        videos_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
        if not os.path.exists(videos_path):
            raise Exception("Metadados de vídeos não encontrados. Execute a CÉLULA 4 primeiro.")

        return True, videos_path
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO PARA ANÁLISE DE CORTES E TRANSIÇÕES: {e}")
        logging.error(f"Pré-requisito não atendido para análise de cortes e transições: {e}")
        return False, None

def detectar_cortes_e_transicoes(video_path, threshold=40.0):
    """Detecta cortes e transições em um vídeo baseado na diferença de frames."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        logging.error(f"Não foi possível abrir o vídeo: {video_path}")
        return []

    cortes_detectados = []
    prev_frame = None
    frame_idx = 0

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        logging.warning(f"FPS do vídeo {video_path} é zero. Não é possível detectar cortes.")
        cap.release()
        return []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        current_frame_gray = cv2.GaussianBlur(current_frame_gray, (5, 5), 0)

        if prev_frame is not None:
            # Calcula a diferença absoluta entre os frames
            diff = cv2.absdiff(current_frame_gray, prev_frame)
            # Calcula a média da diferença
            mean_diff = np.mean(diff)

            # Se a diferença for maior que o threshold, é um corte ou transição
            if mean_diff > threshold:
                timestamp_sec = round(frame_idx / fps, 2)
                cortes_detectados.append({
                    "timestamp_segundos": timestamp_sec,
                    "tipo": "corte_ou_transicao",
                    "diferenca_media": round(float(mean_diff), 2)
                })

        prev_frame = current_frame_gray
        frame_idx += 1

    cap.release()
    return cortes_detectados

def processar_cortes_transicoes_videos(videos_info):
    """Processa a detecção de cortes e transições para todos os vídeos e salva os resultados."""
    analise_visual_dir = os.path.join(PASTA_TRABALHO, "analise_visual")
    os.makedirs(analise_visual_dir, exist_ok=True)

    for i, video_info in enumerate(videos_info):
        video_id = video_info["id"]
        video_path = video_info["caminho_completo"]

        print(f"\nProgresso: {i+1}/{len(videos_info)} - Iniciando detecção de cortes e transições para o vídeo: {video_id} ({video_info["nome_arquivo"]})")
        logging.info(f"Iniciando detecção de cortes e transições para o vídeo: {video_id}")

        cortes_encontrados = detectar_cortes_e_transicoes(video_path)

        output_json_path = os.path.join(analise_visual_dir, f"{video_id}_cortes_transicoes.json")
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump({"video_id": video_id, "cortes_transicoes": cortes_encontrados}, f, indent=2, ensure_ascii=False)
        logging.info(f"Cortes e transições para {video_id} salvos em: {output_json_path}")
        print(f"  ✅ Detecção de cortes e transições concluída para {video_id}. Encontrados {len(cortes_encontrados)} eventos.")
        if cortes_encontrados:
            print(f"  Primeiros 5 eventos: {cortes_encontrados[:5]}")
        else:
            print("  Nenhum corte ou transição significativo detectado.")

    print("\n✅ SUBLAYER 1.1 (TIMING DE CORTES E TRANSIÇÕES) CONCLUÍDA")
    print(f"Todos os resultados de cortes e transições salvos em: {analise_visual_dir}")

# ============================================================================
# INSTRUÇÕES DE EXECUÇÃO NO GOOGLE COLAB:
# 1. Certifique-se de que a CÉLULA 1 (Setup Inicial) foi executada.
# 2. Certifique-se de que a CÉLULA 2 (Configuração Inicial) foi executada e
#    definiu as variáveis globais `PASTA_TRABALHO` e `PASTA_VIDEOS`.
# 3. Certifique-se de que a CÉLULA 4 (Extração de Metadados) foi executada e
#    gerou o arquivo `metadados_completos.json`.
# 4. Copie e cole TODO o conteúdo desta célula em uma NOVA CÉLULA no seu
#    notebook Google Colab, logo APÓS a CÉLULA 5.2 (Ganchos de Dopamina).
# 5. Execute esta nova célula.
# ============================================================================

# Bloco de execução principal para o Colab
if __name__ == '__main__':
    prereq_ok, videos_json_path = verificar_prerequisito_etapa_cortes()
    if prereq_ok:
        try:
            with open(videos_json_path, 'r', encoding='utf-8') as f:
                videos_data = json.load(f)
            processar_cortes_transicoes_videos(videos_data)
        except Exception as e:
            print(f"❌ ERRO GERAL NA SUBLAYER 1.1 (TIMING DE CORTES E TRANSIÇÕES): {e}")
            logging.error(f"Erro geral na Sublayer 1.1 (Timing de Cortes e Transições): {e}")






Progresso: 1/40 - Iniciando detecção de cortes e transições para o vídeo: vid_001 (entenda mais sobre a consciencia cristica.mp4)
  ✅ Detecção de cortes e transições concluída para vid_001. Encontrados 0 eventos.
  Nenhum corte ou transição significativo detectado.

Progresso: 2/40 - Iniciando detecção de cortes e transições para o vídeo: vid_002 (voce quer prosperar.mp4)
  ✅ Detecção de cortes e transições concluída para vid_002. Encontrados 14 eventos.
  Primeiros 5 eventos: [{'timestamp_segundos': 1.63, 'tipo': 'corte_ou_transicao', 'diferenca_media': 63.96}, {'timestamp_segundos': 2.8, 'tipo': 'corte_ou_transicao', 'diferenca_media': 46.39}, {'timestamp_segundos': 3.5, 'tipo': 'corte_ou_transicao', 'diferenca_media': 44.81}, {'timestamp_segundos': 5.1, 'tipo': 'corte_ou_transicao', 'diferenca_media': 68.04}, {'timestamp_segundos': 5.13, 'tipo': 'corte_ou_transicao', 'diferenca_media': 83.39}]

Progresso: 3/40 - Iniciando detecção de cortes e transições para o vídeo: vid_003 (Por q

In [ ]:

# ============================================================================
# CÉLULA 4.8: SUBLAYER 1.2 - RITMO GERAL DO VÍDEO
# Esta célula deve ser executada APÓS as CÉLULA 4 (Metadados),
# CÉLULA 4.4 (Análise de Silêncios) e CÉLULA 4.7 (Timing de Cortes e Transições).
# ============================================================================

# Importar bibliotecas necessárias
import os
import json
import logging

def verificar_prerequisito_etapa_ritmo():
    """Verifica se as etapas anteriores (metadados, cortes/transições, silêncios) foram executadas com sucesso."""
    try:
        if not (
            'PASTA_TRABALHO' in globals() and
            'PASTA_VIDEOS' in globals()
        ):
            raise Exception("As variáveis globais PASTA_TRABALHO e PASTA_VIDEOS não estão definidas. Execute a CÉLULA 2 primeiro.")

        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        if not config['status_etapas']['metadados']:
            raise Exception("A etapa de metadados não foi concluída. Execute a CÉLULA 4 primeiro.")

        videos_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
        if not os.path.exists(videos_path):
            raise Exception("Metadados de vídeos não encontrados. Execute a CÉLULA 4 primeiro.")

        # Verificar se os resultados de cortes/transições e silêncios existem
        analise_visual_dir = os.path.join(PASTA_TRABALHO, "analise_visual")
        analise_audio_dir = os.path.join(PASTA_TRABALHO, "analise_audio")

        # Não é um erro fatal se os arquivos não existirem para todos os vídeos, mas é bom avisar
        # A lógica abaixo vai tentar carregar o que estiver disponível

        return True, videos_path
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO PARA ANÁLISE DE RITMO DO VÍDEO: {e}")
        logging.error(f"Pré-requisito não atendido para análise de ritmo do vídeo: {e}")
        return False, None

def analisar_ritmo_video(video_id, duracao_video, cortes_transicoes, silencios):
    """Analisa o ritmo geral do vídeo com base em cortes/transições e silêncios."""
    ritmo_info = {
        "video_id": video_id,
        "duracao_total_segundos": duracao_video,
        "total_cortes_transicoes": len(cortes_transicoes),
        "densidade_cortes_por_minuto": 0,
        "total_periodos_silencio": len(silencios),
        "duracao_total_silencio_segundos": 0,
        "percentual_silencio": 0,
        "ritmo_inferido": ""
    }

    if duracao_video > 0:
        ritmo_info["densidade_cortes_por_minuto"] = round(len(cortes_transicoes) / (duracao_video / 60), 2)

    total_silencio_segundos = sum([s["duracao_segundos"] for s in silencios])
    ritmo_info["duracao_total_silencio_segundos"] = round(total_silencio_segundos, 2)
    if duracao_video > 0:
        ritmo_info["percentual_silencio"] = round((total_silencio_segundos / duracao_video) * 100, 2)

    # Inferência básica do ritmo
    if ritmo_info["densidade_cortes_por_minuto"] > 15 and ritmo_info["percentual_silencio"] < 10:
        ritmo_info["ritmo_inferido"] = "Muito Rápido e Dinâmico"
    elif ritmo_info["densidade_cortes_por_minuto"] > 8 and ritmo_info["percentual_silencio"] < 20:
        ritmo_info["ritmo_inferido"] = "Rápido e Envolvente"
    elif ritmo_info["densidade_cortes_por_minuto"] > 3 and ritmo_info["percentual_silencio"] < 40:
        ritmo_info["ritmo_inferido"] = "Moderado e Informativo"
    else:
        ritmo_info["ritmo_inferido"] = "Lento e Reflexivo"

    return ritmo_info

def processar_ritmo_videos(videos_info):
    """Processa a análise de ritmo para todos os vídeos e salva os resultados."""
    analise_visual_dir = os.path.join(PASTA_TRABALHO, "analise_visual")
    os.makedirs(analise_visual_dir, exist_ok=True)

    analise_audio_dir = os.path.join(PASTA_TRABALHO, "analise_audio")

    for i, video_info in enumerate(videos_info):
        video_id = video_info["id"]
        video_path = video_info["caminho_completo"]
        duracao_video = video_info.get("duracao_segundos", 0) # Obtém a duração da CÉLULA 4

        print(f"\nProgresso: {i+1}/{len(videos_info)} - Iniciando análise de ritmo para o vídeo: {video_id} ({video_info["nome_arquivo"]})")
        logging.info(f"Iniciando análise de ritmo para o vídeo: {video_id}")

        # Carregar dados de cortes/transições
        cortes_transicoes = []
        cortes_path = os.path.join(analise_visual_dir, f"{video_id}_cortes_transicoes.json")
        if os.path.exists(cortes_path):
            try:
                with open(cortes_path, 'r', encoding='utf-8') as f:
                    cortes_data = json.load(f)
                cortes_transicoes = cortes_data.get("cortes_transicoes", [])
                logging.info(f"Cortes/Transições carregados para {video_id}.")
            except Exception as e:
                logging.error(f"Erro ao carregar cortes/transições para {video_id}: {e}")

        # Carregar dados de silêncios
        silencios = []
        silencios_path = os.path.join(analise_audio_dir, f"{video_id}_silencios.json")
        if os.path.exists(silencios_path):
            try:
                with open(silencios_path, 'r', encoding='utf-8') as f:
                    silencios_data = json.load(f)
                silencios = silencios_data.get("silencios", [])
                logging.info(f"Silêncios carregados para {video_id}.")
            except Exception as e:
                logging.error(f"Erro ao carregar silêncios para {video_id}: {e}")

        ritmo_analise = analisar_ritmo_video(video_id, duracao_video, cortes_transicoes, silencios)

        output_json_path = os.path.join(analise_visual_dir, f"{video_id}_ritmo_geral.json")
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(ritmo_analise, f, indent=2, ensure_ascii=False)
        logging.info(f"Ritmo geral para {video_id} salvo em: {output_json_path}")
        print(f"  ✅ Análise de ritmo concluída para {video_id}.")
        print(f"  Ritmo Inferido: {ritmo_analise['ritmo_inferido']}")
        print(f"  Densidade de Cortes: {ritmo_analise['densidade_cortes_por_minuto']} cortes/min")
        print(f"  Percentual de Silêncio: {ritmo_analise['percentual_silencio']}%")

    print("\n✅ SUBLAYER 1.2 (RITMO GERAL DO VÍDEO) CONCLUÍDA")
    print(f"Todos os resultados de ritmo geral salvos em: {analise_visual_dir}")

# ============================================================================
# INSTRUÇÕES DE EXECUÇÃO NO GOOGLE COLAB:
# 1. Certifique-se de que a CÉLULA 1 (Setup Inicial) foi executada.
# 2. Certifique-se de que a CÉLULA 2 (Configuração Inicial) foi executada e
#    definiu as variáveis globais `PASTA_TRABALHO` e `PASTA_VIDEOS`.
# 3. Certifique-se de que a CÉLULA 4 (Extração de Metadados) foi executada e
#    gerou o arquivo `metadados_completos.json` (que contém a duração do vídeo).
# 4. Certifique-se de que a CÉLULA 4.4 (Análise de Silêncios) e a CÉLULA 4.7
#    (Timing de Cortes e Transições) foram executadas e geraram seus respectivos
#    arquivos de resultados.
# 5. Copie e cole TODO o conteúdo desta célula em uma NOVA CÉLULA no seu
#    notebook Google Colab, logo APÓS a CÉLULA 4.7.
# 6. Execute esta nova célula.
# ============================================================================

# Bloco de execução principal para o Colab
if __name__ == '__main__':
    prereq_ok, videos_json_path = verificar_prerequisito_etapa_ritmo()
    if prereq_ok:
        try:
            with open(videos_json_path, 'r', encoding='utf-8') as f:
                videos_data = json.load(f)
            processar_ritmo_videos(videos_data)
        except Exception as e:
            print(f"❌ ERRO GERAL NA SUBLAYER 1.2 (RITMO GERAL DO VÍDEO): {e}")
            logging.error(f"Erro geral na Sublayer 1.2 (Ritmo Geral do Vídeo): {e}")






Progresso: 1/40 - Iniciando análise de ritmo para o vídeo: vid_001 (entenda mais sobre a consciencia cristica.mp4)
  ✅ Análise de ritmo concluída para vid_001.
  Ritmo Inferido: Lento e Reflexivo
  Densidade de Cortes: 0.0 cortes/min
  Percentual de Silêncio: 0.0%

Progresso: 2/40 - Iniciando análise de ritmo para o vídeo: vid_002 (voce quer prosperar.mp4)
  ✅ Análise de ritmo concluída para vid_002.
  Ritmo Inferido: Muito Rápido e Dinâmico
  Densidade de Cortes: 15.87 cortes/min
  Percentual de Silêncio: 0.0%

Progresso: 3/40 - Iniciando análise de ritmo para o vídeo: vid_003 (Por que é que muitos cristãos não prosperam financeiramente_ não acessam uma ruptura financeira.mp4)
  ✅ Análise de ritmo concluída para vid_003.
  Ritmo Inferido: Lento e Reflexivo
  Densidade de Cortes: 0.0 cortes/min
  Percentual de Silêncio: 0.0%

Progresso: 4/40 - Iniciando análise de ritmo para o vídeo: vid_004 (como é que eu sei que algo que eu estou carregando é da parte de Deus Por dois motivos.

In [ ]:

# ============================================================================
# CÉLULA 4.9: SUBLAYER 2.1 - ANÁLISE DE ÂNGULOS DE CÂMERA E ZOOMS
# Esta célula deve ser executada APÓS a CÉLULA 4 (Extração de Metadados).
# ============================================================================

# Importar bibliotecas necessárias
import os
import json
import logging
import cv2
import numpy as np

def verificar_prerequisito_etapa_camera_zoom():
    """Verifica se a extração de metadados foi executada com sucesso."""
    try:
        if not (
            'PASTA_TRABALHO' in globals() and
            'PASTA_VIDEOS' in globals()
        ):
            raise Exception("As variáveis globais PASTA_TRABALHO e PASTA_VIDEOS não estão definidas. Execute a CÉLULA 2 primeiro.")

        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        if not config['status_etapas']['metadados']:
            raise Exception("A etapa de metadados não foi concluída. Execute a CÉLULA 4 primeiro.")

        videos_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
        if not os.path.exists(videos_path):
            raise Exception("Metadados de vídeos não encontrados. Execute a CÉLULA 4 primeiro.")

        return True, videos_path
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO PARA ANÁLISE DE ÂNGULOS DE CÂMERA E ZOOMS: {e}")
        logging.error(f"Pré-requisito não atendido para análise de ângulos de câmera e zooms: {e}")
        return False, None

def analisar_movimento_camera_zoom(video_path, threshold_movimento=0.01, threshold_zoom=0.005):
    """Analisa o movimento da câmera e o zoom em um vídeo."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        logging.error(f"Não foi possível abrir o vídeo: {video_path}")
        return []

    analise_movimento = []
    prev_frame_gray = None
    frame_idx = 0

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        logging.warning(f"FPS do vídeo {video_path} é zero. Não é possível analisar movimento/zoom.")
        cap.release()
        return []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        current_frame_gray = cv2.GaussianBlur(current_frame_gray, (5, 5), 0)

        if prev_frame_gray is not None:
            # Calcular a diferença absoluta entre os frames para detectar movimento
            diff_frame = cv2.absdiff(current_frame_gray, prev_frame_gray)
            movimento_medio = np.mean(diff_frame) / 255.0 # Normalizar para 0-1

            # Calcular o Optical Flow para detectar movimento direcional e zoom
            # Usamos Lucas-Kanade para pontos esparsos
            # Parâmetros para goodFeaturesToTrack
            feature_params = dict( maxCorners = 100,
                                   qualityLevel = 0.3,
                                   minDistance = 7,
                                   blockSize = 7 )
            # Parâmetros para calcOpticalFlowPyrLK
            lk_params = dict( winSize  = (15,15),
                              maxLevel = 2,
                              criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

            p0 = cv2.goodFeaturesToTrack(prev_frame_gray, mask = None, **feature_params)
            if p0 is not None:
                p1, st, err = cv2.calcOpticalFlowPyrLK(prev_frame_gray, current_frame_gray, p0, None, **lk_params)
                if p1 is not None and st is not None:
                    good_new = p1[st==1]
                    good_old = p0[st==1]

                    if len(good_new) > 0 and len(good_old) > 0:
                        # Calcular o vetor de movimento médio
                        translation_vectors = good_new - good_old
                        mean_translation_x = np.mean(translation_vectors[:, 0])
                        mean_translation_y = np.mean(translation_vectors[:, 1])

                        # Calcular a mudança de escala (zoom)
                        # A mudança na distância entre os pontos pode indicar zoom
                        # Isso é uma simplificação, um método mais robusto usaria homografia
                        scale_change = np.mean(np.linalg.norm(good_new - np.mean(good_new, axis=0), axis=1) /
                                               np.linalg.norm(good_old - np.mean(good_old, axis=0), axis=1))

                        zoom_detectado = ""
                        if scale_change > 1 + threshold_zoom: # Aumentou a escala
                            zoom_detectado = "Zoom In"
                        elif scale_change < 1 - threshold_zoom: # Diminuiu a escala
                            zoom_detectado = "Zoom Out"

                        movimento_camera = ""
                        # Limiares para detectar panning/tilting (ajustar conforme necessário)
                        if abs(mean_translation_x) > 0.5 or abs(mean_translation_y) > 0.5: # Movimento significativo
                            if abs(mean_translation_x) > abs(mean_translation_y) * 2: # Predominantemente horizontal
                                movimento_camera = "Panning" # Movimento horizontal
                            elif abs(mean_translation_y) > abs(mean_translation_x) * 2: # Predominantemente vertical
                                movimento_camera = "Tilting" # Movimento vertical
                            else:
                                movimento_camera = "Movimento Geral"

                        if movimento_camera or zoom_detectado:
                            timestamp_sec = round(frame_idx / fps, 2)
                            analise_movimento.append({
                                "timestamp_segundos": timestamp_sec,
                                "tipo_movimento_camera": movimento_camera,
                                "tipo_zoom": zoom_detectado,
                                "movimento_medio_pixel_diff": round(float(movimento_medio), 4),
                                "escala_alteracao": round(float(scale_change), 4)
                            })

        prev_frame_gray = current_frame_gray
        frame_idx += 1

    cap.release()
    return analise_movimento

def processar_camera_zoom_videos(videos_info):
    """Processa a análise de ângulos de câmera e zooms para todos os vídeos e salva os resultados."""
    analise_visual_dir = os.path.join(PASTA_TRABALHO, "analise_visual")
    os.makedirs(analise_visual_dir, exist_ok=True)

    for i, video_info in enumerate(videos_info):
        video_id = video_info["id"]
        video_path = video_info["caminho_completo"]

        print(f"\nProgresso: {i+1}/{len(videos_info)} - Iniciando análise de ângulos de câmera e zooms para o vídeo: {video_id} ({video_info["nome_arquivo"]})")
        logging.info(f"Iniciando análise de ângulos de câmera e zooms para o vídeo: {video_id}")

        movimentos_encontrados = analisar_movimento_camera_zoom(video_path)

        output_json_path = os.path.join(analise_visual_dir, f"{video_id}_camera_zoom_analise.json")
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump({"video_id": video_id, "movimentos_camera_zoom": movimentos_encontrados}, f, indent=2, ensure_ascii=False)
        logging.info(f"Análise de câmera e zoom para {video_id} salva em: {output_json_path}")
        print(f"  ✅ Análise de ângulos de câmera e zooms concluída para {video_id}. Encontrados {len(movimentos_encontrados)} eventos.")
        if movimentos_encontrados:
            print(f"  Primeiros 5 eventos: {movimentos_encontrados[:5]}")
        else:
            print("  Nenhum movimento de câmera ou zoom significativo detectado.")

    print("\n✅ SUBLAYER 2.1 (ANÁLISE DE ÂNGULOS DE CÂMERA E ZOOMS) CONCLUÍDA")
    print(f"Todos os resultados de análise de câmera e zoom salvos em: {analise_visual_dir}")

# ============================================================================
# INSTRUÇÕES DE EXECUÇÃO NO GOOGLE COLAB:
# 1. Certifique-se de que a CÉLULA 1 (Setup Inicial) foi executada.
# 2. Certifique-se de que a CÉLULA 2 (Configuração Inicial) foi executada e
#    definiu as variáveis globais `PASTA_TRABALHO` e `PASTA_VIDEOS`.
# 3. Certifique-se de que a CÉLULA 4 (Extração de Metadados) foi executada e
#    gerou o arquivo `metadados_completos.json`.
# 4. Copie e cole TODO o conteúdo desta célula em uma NOVA CÉLULA no seu
#    notebook Google Colab, logo APÓS a CÉLULA 4.8 (Ritmo Geral do Vídeo).
# 5. Execute esta nova célula.
# ============================================================================

# Bloco de execução principal para o Colab
if __name__ == '__main__':
    prereq_ok, videos_json_path = verificar_prerequisito_etapa_camera_zoom()
    if prereq_ok:
        try:
            with open(videos_json_path, 'r', encoding='utf-8') as f:
                videos_data = json.load(f)
            processar_camera_zoom_videos(videos_data)
        except Exception as e:
            print(f"❌ ERRO GERAL NA SUBLAYER 2.1 (ANÁLISE DE ÂNGULOS DE CÂMERA E ZOOMS): {e}")
            logging.error(f"Erro geral na Sublayer 2.1 (Análise de Ângulos de Câmera e Zooms): {e}")




Progresso: 1/40 - Iniciando análise de ângulos de câmera e zooms para o vídeo: vid_001 (entenda mais sobre a consciencia cristica.mp4)
  ✅ Análise de ângulos de câmera e zooms concluída para vid_001. Encontrados 124 eventos.
  Primeiros 5 eventos: [{'timestamp_segundos': 0.03, 'tipo_movimento_camera': 'Movimento Geral', 'tipo_zoom': 'Zoom In', 'movimento_medio_pixel_diff': 0.0626, 'escala_alteracao': 1.009}, {'timestamp_segundos': 2.53, 'tipo_movimento_camera': 'Movimento Geral', 'tipo_zoom': 'Zoom In', 'movimento_medio_pixel_diff': 0.0151, 'escala_alteracao': 1.0285}, {'timestamp_segundos': 3.03, 'tipo_movimento_camera': 'Panning', 'tipo_zoom': 'Zoom Out', 'movimento_medio_pixel_diff': 0.016, 'escala_alteracao': 0.9869}, {'timestamp_segundos': 3.5, 'tipo_movimento_camera': '', 'tipo_zoom': 'Zoom In', 'movimento_medio_pixel_diff': 0.0216, 'escala_alteracao': 1.0117}, {'timestamp_segundos': 4.2, 'tipo_movimento_camera': 'Movimento Geral', 'tipo_zoom': 'Zoom Out', 'movimento_medio_pixel

In [ ]:

# ============================================================================
# CÉLULA 4.10: SUBLAYER 2.2 - ANÁLISE DE TRANSIÇÕES E EFEITOS VISUAIS
# Esta célula deve ser executada APÓS a CÉLULA 4 (Extração de Metadados).
# ============================================================================

# Importar bibliotecas necessárias
import os
import json
import logging
import cv2
import numpy as np

def verificar_prerequisito_etapa_transicoes_efeitos():
    """Verifica se a extração de metadados foi executada com sucesso."""
    try:
        if not (
            'PASTA_TRABALHO' in globals() and
            'PASTA_VIDEOS' in globals()
        ):
            raise Exception("As variáveis globais PASTA_TRABALHO e PASTA_VIDEOS não estão definidas. Execute a CÉLULA 2 primeiro.")

        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        if not config['status_etapas']['metadados']:
            raise Exception("A etapa de metadados não foi concluída. Execute a CÉLULA 4 primeiro.")

        videos_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
        if not os.path.exists(videos_path):
            raise Exception("Metadados de vídeos não encontrados. Execute a CÉLULA 4 primeiro.")

        return True, videos_path
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO PARA ANÁLISE DE TRANSIÇÕES E EFEITOS VISUAIS: {e}")
        logging.error(f"Pré-requisito não atendido para análise de transições e efeitos visuais: {e}")
        return False, None

def analisar_transicoes_efeitos(video_path, threshold_diff=30.0, threshold_motion=0.05):
    """Analisa transições e efeitos visuais em um vídeo.
    Esta é uma abordagem simplificada. Transições complexas e efeitos
    exigiriam modelos de ML ou algoritmos mais avançados.
    Aqui, focamos em mudanças visuais abruptas e movimento intenso.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        logging.error(f"Não foi possível abrir o vídeo: {video_path}")
        return []

    transicoes_efeitos_detectados = []
    prev_frame_gray = None
    frame_idx = 0

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        logging.warning(f"FPS do vídeo {video_path} é zero. Não é possível analisar transições/efeitos.")
        cap.release()
        return []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        current_frame_gray = cv2.GaussianBlur(current_frame_gray, (5, 5), 0)

        if prev_frame_gray is not None:
            # Diferença de pixel para detectar mudanças abruptas (cortes ou transições simples)
            diff_frame = cv2.absdiff(current_frame_gray, prev_frame_gray)
            mean_diff = np.mean(diff_frame)

            # Detecção de movimento intenso (pode indicar efeitos visuais ou transições rápidas)
            # Usamos a variância da diferença para capturar a intensidade da mudança
            motion_intensity = np.var(diff_frame)

            timestamp_sec = round(frame_idx / fps, 2)

            # Critérios para transições/efeitos
            if mean_diff > threshold_diff: # Mudança abrupta de cena
                transicoes_efeitos_detectados.append({
                    "timestamp_segundos": timestamp_sec,
                    "tipo": "transicao_abrupta_ou_corte",
                    "detalhe": f"Diferença média de pixels: {round(float(mean_diff), 2)}"
                })
            elif motion_intensity > threshold_motion * 10000: # Movimento intenso (limiar ajustado)
                transicoes_efeitos_detectados.append({
                    "timestamp_segundos": timestamp_sec,
                    "tipo": "efeito_visual_ou_transicao_rapida",
                    "detalhe": f"Intensidade de movimento: {round(float(motion_intensity), 2)}"
                })

        prev_frame_gray = current_frame_gray
        frame_idx += 1

    cap.release()
    return transicoes_efeitos_detectados

def processar_transicoes_efeitos_videos(videos_info):
    """Processa a análise de transições e efeitos visuais para todos os vídeos e salva os resultados."""
    analise_visual_dir = os.path.join(PASTA_TRABALHO, "analise_visual")
    os.makedirs(analise_visual_dir, exist_ok=True)

    for i, video_info in enumerate(videos_info):
        video_id = video_info["id"]
        video_path = video_info["caminho_completo"]

        print(f"\nProgresso: {i+1}/{len(videos_info)} - Iniciando análise de transições e efeitos visuais para o vídeo: {video_id} ({video_info["nome_arquivo"]})")
        logging.info(f"Iniciando análise de transições e efeitos visuais para o vídeo: {video_id}")

        efeitos_encontrados = analisar_transicoes_efeitos(video_path)

        output_json_path = os.path.join(analise_visual_dir, f"{video_id}_transicoes_efeitos.json")
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump({"video_id": video_id, "transicoes_efeitos": efeitos_encontrados}, f, indent=2, ensure_ascii=False)
        logging.info(f"Análise de transições e efeitos para {video_id} salva em: {output_json_path}")
        print(f"  ✅ Análise de transições e efeitos visuais concluída para {video_id}. Encontrados {len(efeitos_encontrados)} eventos.")
        if efeitos_encontrados:
            print(f"  Primeiros 5 eventos: {efeitos_encontrados[:5]}")
        else:
            print("  Nenhuma transição ou efeito visual significativo detectado.")

    print("\n✅ SUBLAYER 2.2 (ANÁLISE DE TRANSIÇÕES E EFEITOS VISUAIS) CONCLUÍDA")
    print(f"Todos os resultados de transições e efeitos visuais salvos em: {analise_visual_dir}")

# ============================================================================
# INSTRUÇÕES DE EXECUÇÃO NO GOOGLE COLAB:
# 1. Certifique-se de que a CÉLULA 1 (Setup Inicial) foi executada.
# 2. Certifique-se de que a CÉLULA 2 (Configuração Inicial) foi executada e
#    definiu as variáveis globais `PASTA_TRABALHO` e `PASTA_VIDEOS`.
# 3. Certifique-se de que a CÉLULA 4 (Extração de Metadados) foi executada e
#    gerou o arquivo `metadados_completos.json`.
# 4. Copie e cole TODO o conteúdo desta célula em uma NOVA CÉLULA no seu
#    notebook Google Colab, logo APÓS a CÉLULA 4.9 (Análise de Ângulos de Câmera e Zooms).
# 5. Execute esta nova célula.
# ============================================================================

# Bloco de execução principal para o Colab
if __name__ == '__main__':
    prereq_ok, videos_json_path = verificar_prerequisito_etapa_transicoes_efeitos()
    if prereq_ok:
        try:
            with open(videos_json_path, 'r', encoding='utf-8') as f:
                videos_data = json.load(f)
            processar_transicoes_efeitos_videos(videos_data)
        except Exception as e:
            print(f"❌ ERRO GERAL NA SUBLAYER 2.2 (ANÁLISE DE TRANSIÇÕES E EFEITOS VISUAIS): {e}")
            logging.error(f"Erro geral na Sublayer 2.2 (Análise de Transições e Efeitos Visuais): {e}")






Progresso: 1/40 - Iniciando análise de transições e efeitos visuais para o vídeo: vid_001 (entenda mais sobre a consciencia cristica.mp4)
  ✅ Análise de transições e efeitos visuais concluída para vid_001. Encontrados 5 eventos.
  Primeiros 5 eventos: [{'timestamp_segundos': 0.03, 'tipo': 'efeito_visual_ou_transicao_rapida', 'detalhe': 'Intensidade de movimento: 751.14'}, {'timestamp_segundos': 5.53, 'tipo': 'efeito_visual_ou_transicao_rapida', 'detalhe': 'Intensidade de movimento: 709.85'}, {'timestamp_segundos': 6.37, 'tipo': 'efeito_visual_ou_transicao_rapida', 'detalhe': 'Intensidade de movimento: 625.66'}, {'timestamp_segundos': 6.9, 'tipo': 'efeito_visual_ou_transicao_rapida', 'detalhe': 'Intensidade de movimento: 629.38'}, {'timestamp_segundos': 16.27, 'tipo': 'efeito_visual_ou_transicao_rapida', 'detalhe': 'Intensidade de movimento: 539.75'}]

Progresso: 2/40 - Iniciando análise de transições e efeitos visuais para o vídeo: vid_002 (voce quer prosperar.mp4)
  ✅ Análise de tran

In [ ]:
# ============================================================================
# CÉLULA 6: SUBLAYER DE CONSOLIDAÇÃO DE DADOS PARA DASHBOARD
# Esta célula deve ser executada APÓS todas as sublayers anteriores.
# ============================================================================

# Importar bibliotecas necessárias
import os
import json
import logging
import pandas as pd
from datetime import datetime


def converter_tipos_json(obj):
    """Converte tipos numpy/pandas para tipos Python nativos"""
    import numpy as np
    import pandas as pd

    if isinstance(obj, (np.integer, pd.Int64Dtype)):
        return int(obj)
    elif isinstance(obj, (np.floating, pd.Float64Dtype)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: converter_tipos_json(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [converter_tipos_json(item) for item in obj]
    elif pd.isna(obj):
        return None
    return obj

def verificar_prerequisito_consolidacao():
    """Verifica se as etapas anteriores foram executadas com sucesso."""
    try:
        if not (
            'PASTA_TRABALHO' in globals() and
            'PASTA_VIDEOS' in globals()
        ):
            raise Exception("As variáveis globais PASTA_TRABALHO e PASTA_VIDEOS não estão definidas. Execute a CÉLULA 2 primeiro.")

        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        if not config['status_etapas']['metadados']:
            raise Exception("A etapa de metadados não foi concluída. Execute a CÉLULA 4 primeiro.")

        videos_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
        if not os.path.exists(videos_path):
            raise Exception("Metadados de vídeos não encontrados. Execute a CÉLULA 4 primeiro.")

        return True, videos_path
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO PARA CONSOLIDAÇÃO DE DADOS: {e}")
        logging.error(f"Pré-requisito não atendido para consolidação de dados: {e}")
        return False, None

def carregar_dados_video(video_id, pasta_trabalho):
    """Carrega todos os dados de análise para um vídeo específico."""
    dados_video = {
        "video_id": video_id,
        "ocr_dados": [],
        "transcricao_audio": "",
        "picos_audio": [],
        "silencios": [],
        "efeitos_audio": {},
        "bpm": None,
        "emocoes": {},
        "ganchos_dopamina": [],
        "cortes_transicoes": [],
        "ritmo_geral": {},
        "camera_zoom": [],
        "transicoes_efeitos": []
    }

    # Carregar dados de OCR
    ocr_path = os.path.join(pasta_trabalho, "analise_texto", f"{video_id}_ocr_results.json")
    if os.path.exists(ocr_path):
        try:
            with open(ocr_path, 'r', encoding='utf-8') as f:
                dados_video["ocr_dados"] = json.load(f)
        except Exception as e:
            logging.error(f"Erro ao carregar OCR para {video_id}: {e}")

    # Carregar transcrição de áudio
    transcricao_path = os.path.join(pasta_trabalho, "analise_audio", "transcricoes", f"{video_id}_transcricao.txt")
    if os.path.exists(transcricao_path):
        try:
            with open(transcricao_path, 'r', encoding='utf-8') as f:
                dados_video["transcricao_audio"] = f.read()
        except Exception as e:
            logging.error(f"Erro ao carregar transcrição para {video_id}: {e}")

    # Carregar picos de áudio
    picos_path = os.path.join(pasta_trabalho, "analise_audio", f"{video_id}_picos_audio.json")
    if os.path.exists(picos_path):
        try:
            with open(picos_path, 'r', encoding='utf-8') as f:
                picos_data = json.load(f)
                dados_video["picos_audio"] = picos_data.get("picos_audio", [])
        except Exception as e:
            logging.error(f"Erro ao carregar picos de áudio para {video_id}: {e}")

    # Carregar silêncios
    silencios_path = os.path.join(pasta_trabalho, "analise_audio", f"{video_id}_silencios.json")
    if os.path.exists(silencios_path):
        try:
            with open(silencios_path, 'r', encoding='utf-8') as f:
                silencios_data = json.load(f)
                dados_video["silencios"] = silencios_data.get("silencios", [])
        except Exception as e:
            logging.error(f"Erro ao carregar silêncios para {video_id}: {e}")

    # Carregar efeitos de áudio
    efeitos_path = os.path.join(pasta_trabalho, "analise_audio", f"{video_id}_efeitos_audio.json")
    if os.path.exists(efeitos_path):
        try:
            with open(efeitos_path, 'r', encoding='utf-8') as f:
                efeitos_data = json.load(f)
                dados_video["efeitos_audio"] = efeitos_data.get("analise_audio", {})
        except Exception as e:
            logging.error(f"Erro ao carregar efeitos de áudio para {video_id}: {e}")

    # Carregar BPM
    bpm_path = os.path.join(pasta_trabalho, "analise_audio", f"{video_id}_bpm.json")
    if os.path.exists(bpm_path):
        try:
            with open(bpm_path, 'r', encoding='utf-8') as f:
                bpm_data = json.load(f)
                dados_video["bpm"] = bpm_data.get("bpm")
        except Exception as e:
            logging.error(f"Erro ao carregar BPM para {video_id}: {e}")

    # Carregar emoções
    emocoes_path = os.path.join(pasta_trabalho, "analise_psicologica", f"{video_id}_emocoes_detectadas.json")
    if os.path.exists(emocoes_path):
        try:
            with open(emocoes_path, 'r', encoding='utf-8') as f:
                emocoes_data = json.load(f)
                dados_video["emocoes"] = emocoes_data.get("emocoes", {})
        except Exception as e:
            logging.error(f"Erro ao carregar emoções para {video_id}: {e}")

    # Carregar ganchos de dopamina
    ganchos_path = os.path.join(pasta_trabalho, "analise_psicologica", f"{video_id}_ganchos_dopamina.json")
    if os.path.exists(ganchos_path):
        try:
            with open(ganchos_path, 'r', encoding='utf-8') as f:
                ganchos_data = json.load(f)
                dados_video["ganchos_dopamina"] = ganchos_data.get("ganchos_dopamina", [])
        except Exception as e:
            logging.error(f"Erro ao carregar ganchos de dopamina para {video_id}: {e}")

    # Carregar cortes e transições
    cortes_path = os.path.join(pasta_trabalho, "analise_visual", f"{video_id}_cortes_transicoes.json")
    if os.path.exists(cortes_path):
        try:
            with open(cortes_path, 'r', encoding='utf-8') as f:
                cortes_data = json.load(f)
                dados_video["cortes_transicoes"] = cortes_data.get("cortes_transicoes", [])
        except Exception as e:
            logging.error(f"Erro ao carregar cortes e transições para {video_id}: {e}")

    # Carregar ritmo geral
    ritmo_path = os.path.join(pasta_trabalho, "analise_visual", f"{video_id}_ritmo_geral.json")
    if os.path.exists(ritmo_path):
        try:
            with open(ritmo_path, 'r', encoding='utf-8') as f:
                dados_video["ritmo_geral"] = json.load(f)
        except Exception as e:
            logging.error(f"Erro ao carregar ritmo geral para {video_id}: {e}")

    # Carregar análise de câmera e zoom
    camera_path = os.path.join(pasta_trabalho, "analise_visual", f"{video_id}_camera_zoom_analise.json")
    if os.path.exists(camera_path):
        try:
            with open(camera_path, 'r', encoding='utf-8') as f:
                camera_data = json.load(f)
                dados_video["camera_zoom"] = camera_data.get("movimentos_camera_zoom", [])
        except Exception as e:
            logging.error(f"Erro ao carregar análise de câmera para {video_id}: {e}")

    # Carregar transições e efeitos visuais
    efeitos_visuais_path = os.path.join(pasta_trabalho, "analise_visual", f"{video_id}_transicoes_efeitos.json")
    if os.path.exists(efeitos_visuais_path):
        try:
            with open(efeitos_visuais_path, 'r', encoding='utf-8') as f:
                efeitos_data = json.load(f)
                dados_video["transicoes_efeitos"] = efeitos_data.get("transicoes_efeitos", [])
        except Exception as e:
            logging.error(f"Erro ao carregar transições e efeitos visuais para {video_id}: {e}")

    return dados_video

def criar_dataframe_consolidado(videos_info, pasta_trabalho):
    """Cria um DataFrame consolidado com todos os dados de análise."""
    dados_consolidados = []

    for video_info in videos_info:
        video_id = video_info["id"]
        dados_video = carregar_dados_video(video_id, pasta_trabalho)

        # Criar linha consolidada para o DataFrame
        linha = {
            "video_id": video_id,
            "nome_arquivo": video_info.get("nome_arquivo", ""),
            "duracao_segundos": video_info.get("duracao_segundos", 0),
            "fps": video_info.get("fps", 0),
            "resolucao": f"{video_info.get('largura', 0)}x{video_info.get('altura', 0)}",

            # Dados de texto
            "total_texto_ocr": len(dados_video["ocr_dados"]),
            "transcricao_disponivel": bool(dados_video["transcricao_audio"]),
            "tamanho_transcricao": len(dados_video["transcricao_audio"]),

            # Dados de áudio
            "total_picos_audio": len(dados_video["picos_audio"]),
            "total_silencios": len(dados_video["silencios"]),
            "bpm": dados_video["bpm"],
            "tipo_audio_inferido": dados_video["efeitos_audio"].get("tipo_audio_inferido", ""),

            # Dados psicológicos
            "total_emocoes_detectadas": len(dados_video["emocoes"]),
            "emocoes_principais": ", ".join(dados_video["emocoes"].keys()),
            "total_ganchos_dopamina": len(dados_video["ganchos_dopamina"]),
            "categorias_ganchos": ", ".join(set([g.get("categoria", "") for g in dados_video["ganchos_dopamina"]])),

            # Dados visuais
            "total_cortes_transicoes": len(dados_video["cortes_transicoes"]),
            "densidade_cortes_por_minuto": dados_video["ritmo_geral"].get("densidade_cortes_por_minuto", 0),
            "ritmo_inferido": dados_video["ritmo_geral"].get("ritmo_inferido", ""),
            "percentual_silencio": dados_video["ritmo_geral"].get("percentual_silencio", 0),
            "total_movimentos_camera": len(dados_video["camera_zoom"]),
            "total_efeitos_visuais": len(dados_video["transicoes_efeitos"]),

            # Timestamp da análise
            "timestamp_analise": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        dados_consolidados.append(linha)

    return pd.DataFrame(dados_consolidados)

def processar_consolidacao_dados(videos_info):
    """Processa a consolidação de todos os dados de análise."""
    dashboard_dir = os.path.join(PASTA_TRABALHO, "dashboard")
    os.makedirs(dashboard_dir, exist_ok=True)

    print("🔄 Iniciando consolidação de dados para o dashboard...")
    logging.info("Iniciando consolidação de dados para o dashboard")

    # Criar DataFrame consolidado
    df_consolidado = criar_dataframe_consolidado(videos_info, PASTA_TRABALHO)

    # Salvar como CSV
    csv_path = os.path.join(dashboard_dir, "dados_consolidados.csv")
    df_consolidado.to_csv(csv_path, index=False, encoding='utf-8')
    print(f"✅ Dados consolidados salvos em CSV: {csv_path}")

    # Salvar como Excel
    excel_path = os.path.join(dashboard_dir, "dados_consolidados.xlsx")
    df_consolidado.to_excel(excel_path, index=False, engine='openpyxl')
    print(f"✅ Dados consolidados salvos em Excel: {excel_path}")

    # Criar JSON detalhado para cada vídeo
    dados_detalhados = {}
    for video_info in videos_info:
        video_id = video_info["id"]
        dados_video = carregar_dados_video(video_id, PASTA_TRABALHO)
        dados_video.update({
            "metadados": video_info
        })
        dados_detalhados[video_id] = dados_video

    # Salvar JSON detalhado
    json_path = os.path.join(dashboard_dir, "dados_detalhados.json")
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(dados_detalhados, f, indent=2, ensure_ascii=False)
    print(f"✅ Dados detalhados salvos em JSON: {json_path}")

    # Criar estatísticas gerais (versão corrigida)
    estatisticas = {
    "total_videos": int(len(videos_info)),
    "duracao_total_segundos": float(df_consolidado["duracao_segundos"].sum()),
    "duracao_media_segundos": float(df_consolidado["duracao_segundos"].mean()),
    "bpm_medio": float(df_consolidado["bpm"].mean()) if df_consolidado["bpm"].notna().any() else 0.0,
    "densidade_cortes_media": float(df_consolidado["densidade_cortes_por_minuto"].mean()),
    "percentual_silencio_medio": float(df_consolidado["percentual_silencio"].mean()),
    "total_ganchos_dopamina": int(df_consolidado["total_ganchos_dopamina"].sum()),
    "total_emocoes_detectadas": int(df_consolidado["total_emocoes_detectadas"].sum()),
    "ritmos_mais_comuns": {k: int(v) for k, v in df_consolidado["ritmo_inferido"].value_counts().to_dict().items()},
    "tipos_audio_mais_comuns": {k: int(v) for k, v in df_consolidado["tipo_audio_inferido"].value_counts().to_dict().items()},
    "timestamp_consolidacao": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    # Salvar estatísticas
    stats_path = os.path.join(dashboard_dir, "estatisticas_gerais.json")
    with open(stats_path, 'w', encoding='utf-8') as f:
        json.dump(estatisticas, f, indent=2, ensure_ascii=False)
    print(f"✅ Estatísticas gerais salvas em: {stats_path}")

    print("\n✅ CONSOLIDAÇÃO DE DADOS CONCLUÍDA")
    print(f"Todos os arquivos para o dashboard salvos em: {dashboard_dir}")
    print(f"📊 Total de vídeos processados: {len(videos_info)}")
    print(f"📈 Duração total analisada: {df_consolidado['duracao_segundos'].sum():.1f} segundos")

# ============================================================================
# INSTRUÇÕES DE EXECUÇÃO NO GOOGLE COLAB:
# 1. Certifique-se de que TODAS as sublayers anteriores foram executadas com sucesso.
# 2. Certifique-se de que a CÉLULA 1 (Setup Inicial) instalou a biblioteca `openpyxl`:
#    `!pip install -q openpyxl`
# 3. Certifique-se de que a CÉLULA 2 (Configuração Inicial) foi executada e
#    definiu as variáveis globais `PASTA_TRABALHO` e `PASTA_VIDEOS`.
# 4. Copie e cole TODO o conteúdo desta célula em uma NOVA CÉLULA no seu
#    notebook Google Colab, logo APÓS todas as sublayers anteriores.
# 5. Execute esta nova célula.
# ============================================================================

# Bloco de execução principal para o Colab
if __name__ == '__main__':
    prereq_ok, videos_json_path = verificar_prerequisito_consolidacao()
    if prereq_ok:
        try:
            with open(videos_json_path, 'r', encoding='utf-8') as f:
                videos_data = json.load(f)
            processar_consolidacao_dados(videos_data)
        except Exception as e:
            print(f"❌ ERRO GERAL NA CONSOLIDAÇÃO DE DADOS: {e}")
            logging.error(f"Erro geral na consolidação de dados: {e}")




🔄 Iniciando consolidação de dados para o dashboard...
✅ Dados consolidados salvos em CSV: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/dashboard/dados_consolidados.csv
✅ Dados consolidados salvos em Excel: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/dashboard/dados_consolidados.xlsx
✅ Dados detalhados salvos em JSON: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/dashboard/dados_detalhados.json
✅ Estatísticas gerais salvas em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/dashboard/estatisticas_gerais.json

✅ CONSOLIDAÇÃO DE DADOS CONCLUÍDA
Todos os arquivos para o dashboard salvos em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/dashboard
📊 Total de vídeos processados: 40
📈 Duração total analisada: 1750.6 segundos


In [ ]:
# ============================================================================
# CÉLULA 5: DECOMPOSIÇÃO COMPLETA DOS VÍDEOS
# ============================================================================

def verificar_prerequisito_etapa4():
    """Verifica se metadados foram extraídos com sucesso"""
    try:
        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        if not config['status_etapas']['metadados']:
            raise Exception("Execute a CÉLULA 4 primeiro")

        metadados_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
        if not os.path.exists(metadados_path):
            raise Exception("Metadados não encontrados. Execute a CÉLULA 4 primeiro")

        return True, metadados_path
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO: {e}")
        return False, None

def extrair_frames_video(video_path, video_id):
    """Extrai frames do vídeo para análise"""
    frames_dir = os.path.join(PASTA_TRABALHO, "frames_extraidos", video_id)
    os.makedirs(frames_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Extrair 1 frame por segundo
    intervalo = max(1, int(fps))
    frames_salvos = []
    frame_atual = 0

    while frame_atual < total_frames:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_atual)
        ret, frame = cap.read()

        if not ret:
            break

        segundo = frame_atual / fps
        nome_frame = f"frame_{segundo:06.2f}s.jpg"
        path_frame = os.path.join(frames_dir, nome_frame)

        cv2.imwrite(path_frame, frame, [cv2.IMWRITE_JPEG_QUALITY, 85])

        frames_salvos.append({
            'segundo': round(segundo, 2),
            'frame_num': frame_atual,
            'arquivo': nome_frame,
            'path': path_frame
        })

        frame_atual += intervalo

    cap.release()
    return frames_salvos

def detectar_cortes_video(video_path):
    """Detecta cortes/mudanças de cena"""
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    cortes = []
    frame_anterior = None
    frame_num = 0
    threshold = 0.35

    while True:
        ret, frame_atual = cap.read()
        if not ret:
            break

        if frame_anterior is not None:
            gray_atual = cv2.cvtColor(frame_atual, cv2.COLOR_BGR2GRAY)
            gray_anterior = cv2.cvtColor(frame_anterior, cv2.COLOR_BGR2GRAY)

            diff = cv2.absdiff(gray_atual, gray_anterior)
            score = np.mean(diff) / 255.0

            if score > threshold:
                segundo = frame_num / fps
                cortes.append({
                    'segundo': round(segundo, 2),
                    'frame_num': frame_num,
                    'intensidade': round(score, 3)
                })

        frame_anterior = frame_atual.copy()
        frame_num += 1

    cap.release()

    # Filtrar cortes muito próximos
    cortes_filtrados = []
    for corte in cortes:
        if not cortes_filtrados or (corte['segundo'] - cortes_filtrados[-1]['segundo']) >= 1.0:
            cortes_filtrados.append(corte)

    return cortes_filtrados

def processar_ocr_frames(video_id, frames):
    """Processa OCR em frames selecionados"""
    textos_encontrados = []

    # Processar 1 a cada 3 frames para otimizar
    frames_ocr = frames[::3]

    for frame_info in frames_ocr:
        try:
            img = cv2.imread(frame_info['path'])

            # Preprocessar para OCR
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            enhanced = clahe.apply(gray)

            # Converter para PIL
            img_pil = Image.fromarray(enhanced)

            # OCR
            config_ocr = "--oem 3 --psm 6 -l por+eng"
            texto = pytesseract.image_to_string(img_pil, config=config_ocr)

            if texto.strip():
                textos_encontrados.append({
                    'segundo': frame_info['segundo'],
                    'frame': frame_info['arquivo'],
                    'texto': texto.strip()
                })

        except Exception:
            continue

    return textos_encontrados

def analisar_audio_video(video_path, video_id):
    """Analisa características do áudio"""
    try:
        clip = VideoFileClip(video_path)

        if not clip.audio:
            clip.close()
            return {'tem_audio': False}

        # Extrair áudio temporariamente
        temp_audio = os.path.join(PASTA_TRABALHO, "temp", f"{video_id}_audio.wav")
        clip.audio.write_audiofile(temp_audio, logger=None, verbose=False)

        # Analisar com librosa
        y, sr = librosa.load(temp_audio, sr=22050)

        # Detectar tempo e beats
        tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
        beat_times = librosa.frames_to_time(beats, sr=sr)

        # Análise de energia
        hop_length = 512
        frame_length = 2048
        energy = np.array([
            np.sum(np.abs(y[i:i+frame_length])**2)
            for i in range(0, len(y)-frame_length, hop_length)
        ])

        # Limpeza
        os.remove(temp_audio)
        clip.close()

        return {
            'tem_audio': True,
            'duracao_audio': len(y) / sr,
            'tempo_bpm': float(tempo),
            'total_beats': len(beats),
            'energia_media': float(np.mean(energy)),
            'variacao_energia': float(np.std(energy))
        }

    except Exception as e:
        return {'tem_audio': False, 'erro': str(e)}

def decompor_video_completo(video_info):
    """Decomposição completa de um vídeo"""
    video_path = video_info['caminho_completo']
    video_id = video_info['id']

    # Extrair frames
    frames = extrair_frames_video(video_path, video_id)

    # Detectar cortes
    cortes = detectar_cortes_video(video_path)

    # OCR nos frames
    textos = processar_ocr_frames(video_id, frames)

    # Análise de áudio
    audio_analise = analisar_audio_video(video_path, video_id)

    resultado = {
        'video_id': video_id,
        'video_info': video_info,
        'frames_extraidos': len(frames),
        'frames_detalhes': frames,
        'cortes_detectados': cortes,
        'textos_encontrados': textos,
        'analise_audio': audio_analise,
        'processado_em': datetime.now().isoformat()
    }

    return resultado

def processar_decomposicao_completa():
    """Processa decomposição de todos os vídeos"""
    # Carregar metadados
    metadados_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")

    if not os.path.exists(metadados_path):
        print("❌ ERRO: Arquivo de metadados não encontrado")
        print("Execute a CÉLULA 4 primeiro para extrair metadados")
        return [], 0

    with open(metadados_path, 'r', encoding='utf-8') as f:
        videos_metadados = json.load(f)

    # Filtrar apenas vídeos com metadados válidos
    videos_para_processar = [v for v in videos_metadados if v.get('status') == 'metadados_extraidos']

    print(f"Total de vídeos nos metadados: {len(videos_metadados)}")
    print(f"Vídeos com status 'metadados_extraidos': {len(videos_para_processar)}")

    # Debug: mostrar status de todos os vídeos
    status_count = {}
    for v in videos_metadados:
        status = v.get('status', 'sem_status')
        status_count[status] = status_count.get(status, 0) + 1

    print(f"Status dos vídeos: {status_count}")

    if not videos_para_processar:
        print("❌ NENHUM VÍDEO COM STATUS 'metadados_extraidos' ENCONTRADO")
        print("Verifique se a CÉLULA 4 foi executada com sucesso")
        return [], 0

    resultados_decomposicao = []
    sucessos = 0

    print(f"Processando decomposição de {len(videos_para_processar)} vídeos...")

    for i, video in enumerate(videos_para_processar, 1):
        print(f"[{i}/{len(videos_para_processar)}] {video['nome_arquivo']}")

        try:
            resultado = decompor_video_completo(video)
            resultados_decomposicao.append(resultado)
            sucessos += 1

            # Mostrar progresso
            frames = resultado['frames_extraidos']
            cortes = len(resultado['cortes_detectados'])
            textos = len(resultado['textos_encontrados'])
            audio = "Sim" if resultado['analise_audio']['tem_audio'] else "Não"

            print(f"  ✓ Frames: {frames} | Cortes: {cortes} | Textos: {textos} | Áudio: {audio}")

        except Exception as e:
            print(f"  ❌ Erro: {e}")
            continue

    return resultados_decomposicao, sucessos

def salvar_decomposicao(resultados):
    """Salva resultados da decomposição"""
    decomp_path = os.path.join(PASTA_TRABALHO, "dados", "decomposicao_completa.json")
    with open(decomp_path, 'w', encoding='utf-8') as f:
        json.dump(resultados, f, indent=2, ensure_ascii=False)

    # Atualizar config
    config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
    with open(config_path, 'r', encoding='utf-8') as f:
        config = json.load(f)

    config['status_etapas']['decomposicao'] = True
    config['videos_decompostos'] = len(resultados)

    with open(config_path, 'w', encoding='utf-8') as f:
        json.dump(config, f, indent=2, ensure_ascii=False)

    return decomp_path

# Executar decomposição
prerequisito_ok, metadados_path = verificar_prerequisito_etapa4()

if prerequisito_ok:
    try:
        resultados, sucessos = processar_decomposicao_completa()

        if sucessos == 0:
            print("❌ NENHUMA DECOMPOSIÇÃO REALIZADA COM SUCESSO")
        else:
            decomp_path = salvar_decomposicao(resultados)

            print("✅ DECOMPOSIÇÃO CONCLUÍDA")
            print(f"Vídeos processados: {sucessos}")
            print(f"Dados salvos em: {decomp_path}")

            # Resumo
            total_frames = sum(r['frames_extraidos'] for r in resultados)
            total_cortes = sum(len(r['cortes_detectados']) for r in resultados)
            total_textos = sum(len(r['textos_encontrados']) for r in resultados)

            print(f"\nResumo da decomposição:")
            print(f"Total frames extraídos: {total_frames}")
            print(f"Total cortes detectados: {total_cortes}")
            print(f"Total textos encontrados: {total_textos}")
            print("Próximo passo: Execute a CÉLULA 6 - ANÁLISE DE PADRÕES")

    except Exception as e:
        print(f"❌ ERRO NA DECOMPOSIÇÃO: {e}")

Total de vídeos nos metadados: 40
Vídeos com status 'metadados_extraidos': 0
Status dos vídeos: {'descoberto': 40}
❌ NENHUM VÍDEO COM STATUS 'metadados_extraidos' ENCONTRADO
Verifique se a CÉLULA 4 foi executada com sucesso
❌ NENHUMA DECOMPOSIÇÃO REALIZADA COM SUCESSO
